In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import pandas as pd
import os
import glob
import numpy as np

<IPython.core.display.Javascript object>

In [3]:
import warnings

# Suppress specific FutureWarning
warnings.filterwarnings("ignore", category=FutureWarning, module="seaborn._oldcore")


<IPython.core.display.Javascript object>

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns

<IPython.core.display.Javascript object>

In [5]:
from sklearn.preprocessing import StandardScaler

<IPython.core.display.Javascript object>

In [6]:
from collections import deque, defaultdict


<IPython.core.display.Javascript object>

In [7]:
def read_csv_files_grouped(csv_files, plant, header=[0, 1]):
    results = []
    for filepath in csv_files:
        df = pd.read_csv(filepath, header=header, low_memory=False)
        df["plant"] = plant  # Add the 'plant' column
        results.append(df)
    df = pd.concat(results, ignore_index=True)
    return df

<IPython.core.display.Javascript object>

In [8]:
def read_csv_files_full(csv_files, plant):
    results = []
    for filepath in csv_files:
        df = pd.read_csv(filepath, low_memory=False)
        df["plant"] = plant  # Add the 'plant' column
        results.append(df)
    df = pd.concat(results, ignore_index=True)
    return df

<IPython.core.display.Javascript object>

### Renaming the Columns

In [9]:
def preprocess_rename_columns(df):
    df.columns = pd.MultiIndex.from_tuples(
        list(
            {
                col: (col[0], "") if "Unnamed" in col[1] else col for col in df.columns
            }.values()
        )
    )
    return df

<IPython.core.display.Javascript object>

### Changing the order_of columns Columns

In [10]:
def preprocess_change_columns_order(df, column, pos):
    # Get the list of columns
    cols = df.columns.tolist()
    # Remove the specified column
    cols.remove(column)
    # Insert the column at the desired position
    cols.insert(pos, column)
    # Reorder the DataFrame columns
    return df[cols]

<IPython.core.display.Javascript object>

In [11]:
def read_csv_files_path(csv_files_path_dict, path, plant):
    csv_files = glob.glob(os.path.join(path, "*.csv"))
    csv_files_path_dict[plant] = csv_files
    return csv_files_path_dict

<IPython.core.display.Javascript object>

# Topological Analysis - NEW

## Definitions

In [12]:
# Function to compare two models strictly based on means only
def dominates(model1, model2):
    """Return True if model1 strictly dominates model2, False otherwise."""
    # Check if model1 is at least as good as model2 in all metrics
    at_least_as_good = (
        model1["RMSE_mean"] <= model2["RMSE_mean"]
        and model1["MAE_mean"] <= model2["MAE_mean"]
        and model1["MAPE_mean"] <= model2["MAPE_mean"]
        and model1["R2_mean"] >= model2["R2_mean"]
    )

    # Check if model1 is strictly better in at least one metric
    strictly_better_in_one = (
        model1["RMSE_mean"] < model2["RMSE_mean"]
        or model1["MAE_mean"] < model2["MAE_mean"]
        or model1["MAPE_mean"] < model2["MAPE_mean"]
        or model1["R2_mean"] > model2["R2_mean"]
    )

    # Model1 strictly dominates Model2 if it is at least as good in all metrics and
    # strictly better in at least one metric
    return at_least_as_good and strictly_better_in_one

<IPython.core.display.Javascript object>

In [13]:
def get_dominance_matrix_and_graph(df):
    n = len(df)
    dominance_matrix = np.zeros((n, n), dtype=bool)

    for i in range(n):
        for j in range(n):
            if i != j:
                dominance_matrix[i, j] = dominates(df.iloc[i], df.iloc[j])

    # Create the dominance graph
    dominance_graph = {i: [] for i in range(n)}
    for i in range(n):
        for j in range(n):
            if dominance_matrix[i, j]:
                dominance_graph[i].append(j)

    return dominance_matrix, dominance_graph

<IPython.core.display.Javascript object>

In [14]:
# Topological Sorting
def topological_sort(graph):
    """Perform topological sorting on the given graph."""
    in_degree = {u: 0 for u in graph}
    for u in graph:
        for v in graph[u]:
            in_degree[v] += 1

    queue = deque([u for u in graph if in_degree[u] == 0])
    topo_order = []

    while queue:
        u = queue.popleft()
        topo_order.append(u)

        for v in graph[u]:
            in_degree[v] -= 1
            if in_degree[v] == 0:
                queue.append(v)

    return topo_order

<IPython.core.display.Javascript object>

In [15]:
# Function to find the top models in each group
def find_top_models(group):
    max_net_dominance = group["Net_Dominance"].max()
    top_models = group[group["Net_Dominance"] == max_net_dominance]
    return top_models

<IPython.core.display.Javascript object>

### Dominance Analysis and SCPM

In [16]:
def compute_scpm(df):
    scaler = StandardScaler()
    standardized_metrics = scaler.fit_transform(
        df[["RMSE_mean", "MAE_mean", "MAPE_mean", "R2_mean"]]
    )

    # Creating a new dataframe with standardized values
    standardized_df = pd.DataFrame(
        standardized_metrics,
        columns=["RMSE_mean", "MAE_mean", "MAPE_mean", "R2_mean"],
    )

    # Summing all metrics and subtracting R2_mean
    standardized_df["Result"] = (
        standardized_df[["RMSE_mean", "MAE_mean", "MAPE_mean"]].sum(axis=1)
        - standardized_df["R2_mean"]
    )

    df["SCPM"] = standardized_df["Result"]
    return df

<IPython.core.display.Javascript object>

In [17]:
def make_dominance_analysis(df):
    # Compute dominance matrix and graph
    dominance_matrix, dominance_graph = get_dominance_matrix_and_graph(df)

    # Get the topological order
    topo_order = topological_sort(dominance_graph)
    df = df.copy()
    n = len(topo_order)

    # Identify Dominant Models
    dominant_models = [
        df["Model_bkp_2"].iloc[i]
        for i in range(len(df))
        if not np.any(dominance_matrix[:, i])
    ]

    # Identify Non-Dominant Models
    non_dominant_models = [
        df["Model_bkp_2"].iloc[i]
        for i in range(len(df))
        if np.any(dominance_matrix[:, i])
    ]

    # Identify Models that are both dominated and dominate at least one
    intermediate_models = [
        df["Model_bkp_2"].iloc[i]
        for i in range(len(df))
        if np.any(dominance_matrix[:, i]) and np.any(dominance_matrix[i, :])
    ]

    # Add count of times each model is dominated by others
    df["Dominated_Count"] = np.sum(dominance_matrix, axis=0)

    # Classify models
    df["Classification"] = "intermediate_model"  # default to intermediate_model
    df.loc[
        ~np.any(dominance_matrix, axis=1) & np.any(dominance_matrix, axis=0),
        "Classification",
    ] = "non_dominant_model"
    df.loc[~np.any(dominance_matrix, axis=0), "Classification"] = "dominant_model"

    # Map the topological order to the model names and get dominance counts
    sorted_models = [df["Model"].iloc[i] for i in topo_order]

    df_sorted = df.iloc[topo_order].copy()
    df_sorted["Dominates_Count"] = dominance_matrix.sum(axis=1)[topo_order]
    df_sorted_topo = df_sorted.copy()

    df_sorted = df_sorted.sort_values(by="Dominates_Count", ascending=False)
    df_sorted_count = df_sorted.copy()

    return {
        "dominance_matrix": dominance_matrix,
        "dominance_graph": dominance_graph,
        "df_sorted_topo": df_sorted_topo,
        "df_sorted_count": df_sorted_count,
    }

<IPython.core.display.Javascript object>

# Pre Training Analysis

# Reading The files

In [18]:
csv_files_path_fine_tuning = dict()
csv_files_path_pre_train = dict()

<IPython.core.display.Javascript object>

## 206

### Plant B

In [19]:
path_pre_train = "/home/peressim/projects/ccs28-ml-modelling/reports/results/global_models/206/b/pre_training/full/"
plant = "b"

csv_files_path_pre_train = read_csv_files_path(
    csv_files_path_pre_train, path_pre_train, plant
)

<IPython.core.display.Javascript object>

### Reading all data - pre training

#### Full

In [20]:
dfs_plant = dict()
column = ("plant", "")
pos = 2

for plant, csv_files in csv_files_path_pre_train.items():
    df = read_csv_files_full(csv_files, plant)
    df["plant"] = plant
    dfs_plant[plant] = df

<IPython.core.display.Javascript object>

In [21]:
df_pre_train = pd.concat(dfs_plant.values(), ignore_index=True)

<IPython.core.display.Javascript object>

In [22]:
df_pre_train.shape

(219, 23)

<IPython.core.display.Javascript object>

## Preprocessing

In [23]:
patterns = [
    "Chemical + Mineralogical + Feature Engineering",
    "Chemical + Mineralogical + CS7 + One-Hot",
    "Chemical + Mineralogical + Physical + One-Hot",
    "Chemical + Mineralogical + CS3 + One-Hot",
    "Chemical + Mineralogical + CS3 + CS7 + One-Hot",
    "Chemical + Mineralogical + One-Hot",
    "Chemical + Mineralogical + CS1 + CS3 + CS7 + One-Hot",
    "Chemical + Mineralogical + CS1 + CS3 + One-Hot",
    "Chemical + Mineralogical + CS1 + One-Hot",
    "Chemical + Mineralogical + CS2 + One-Hot",
    "Chemical + Feature Engineering",
]

FEATURES_TO_REPLACE_1 = {
    "Chemical + Mineralogical + CS3 + CS7": "Chemical + Mineralogical + Early CS",
    "Chemical + Mineralogical + CS7": "Chemical + Mineralogical + Early CS",
    "Chemical + Mineralogical + CS3": "Chemical + Mineralogical + Early CS",
    "Chemical + Mineralogical + CS1": "Chemical + Mineralogical + Early CS",
    "Chemical + Mineralogical + CS1 + CS3": "Chemical + Mineralogical + Early CS",
    "Chemical + Mineralogical + CS1 + CS3 + CS7": "Chemical + Mineralogical + Early CS",
    "Chemical + Mineralogical + CS2": "Chemical + Mineralogical + Early CS",
    "Chemical + CS1 + CS3 + CS7": "Chemical + Mineralogical + Early CS",
    "Chemical + CS3 + CS7": "Chemical + Mineralogical + Early CS",
    "Chemical + CS7": "Chemical + Mineralogical + Early CS",
    "Chemical + CS3": "Chemical + Mineralogical + Early CS",
    "Chemical + CS1 + CS3": "Chemical + Mineralogical + Early CS",
    "Chemical + CS1": "Chemical + Mineralogical + Early CS",
    "Chemical + CS1 + CS7": "Chemical + Mineralogical + Early CS",
}

FEATURES_TO_REPLACE_2 = {
    "Chemical": "Chemical + Mineralogical",
    "Chemical + Properties CS Less": "Chemical + Mineralogical + Physical - Early CS",
    "Chemical + Mineralogical + Properties CS Less": "Chemical + Mineralogical + Physical - Early CS",
    "Chemical + Physical": "Chemical + Mineralogical + Physical",
}

df_pre_train = df_pre_train[
    ~df_pre_train["Features"].apply(lambda x: x in patterns)
].reset_index(drop=True)

df_pre_train = df_pre_train[
    ~df_pre_train["Cross Validation"].eq("Out of time Split")
].reset_index(drop=True)

# removing afterwards
df_pre_train = df_pre_train[
    ~df_pre_train["Cross Validation"].eq("Repeated KFold")
].reset_index(drop=True)

df_pre_train["Features_bkp"] = df_pre_train["Features"].copy()
df_pre_train["Features"] = df_pre_train["Features"].replace(FEATURES_TO_REPLACE_1)

df_pre_train["Features"] = df_pre_train["Features"].replace(FEATURES_TO_REPLACE_2)


# removing afterwards
# df_pre_train = df_pre_train[
#     ~df_pre_train["Features"].eq("Chemical + Mineralogical + Early CS")
# ].reset_index(drop=True)

metrics_names = {
    "RMSE Test": "RMSE_mean",
    "MAE Test": "MAE_mean",
    "MAPE Test": "MAPE_mean",
    "R2 Test": "R2_mean",
}

df_pre_train = df_pre_train.rename(metrics_names, axis=1)

<IPython.core.display.Javascript object>

In [24]:
df_pre_train["Features"].unique()

array(['Chemical + Mineralogical + Physical - Early CS',
       'Chemical + Mineralogical + Physical', 'Chemical + Mineralogical'],
      dtype=object)

<IPython.core.display.Javascript object>

In [25]:
df_pre_train["MAPE Train"] = df_pre_train["MAPE Train"] * 100
df_pre_train["MAPE_mean"] = df_pre_train["MAPE_mean"] * 100

df_pre_train["Model_bkp"] = df_pre_train["Model"].copy()
df_pre_train["Model_bkp_2"] = df_pre_train["Model"].apply(
    lambda x: "" if pd.isna(x) else "_" + str(x)
)

<IPython.core.display.Javascript object>

In [26]:
df_pre_train["Model_bkp_2"] = df_pre_train["Model_bkp_2"].apply(
    lambda x: "MLP"
    if "MLP" in x
    else "Bi-LSTM"
    if "Bi-LSTM" in x
    else "LSTM"
    if "LSTM" in x
    else "Conv1D"
    if "Conv1D" in x
    else "Transformer"
)

<IPython.core.display.Javascript object>

In [27]:
df_pre_train["Cross Validation"].unique()

array(['Out of time'], dtype=object)

<IPython.core.display.Javascript object>

In [28]:
df_pre_train["Features"].unique()

array(['Chemical + Mineralogical + Physical - Early CS',
       'Chemical + Mineralogical + Physical', 'Chemical + Mineralogical'],
      dtype=object)

<IPython.core.display.Javascript object>

In [29]:
df_pre_train["Features_bkp"].unique()

array(['Chemical + Properties CS Less', 'Chemical + Physical', 'Chemical'],
      dtype=object)

<IPython.core.display.Javascript object>

In [30]:
df_pre_train.shape

(219, 26)

<IPython.core.display.Javascript object>

In [31]:
df_pre_train["Features"].unique()

array(['Chemical + Mineralogical + Physical - Early CS',
       'Chemical + Mineralogical + Physical', 'Chemical + Mineralogical'],
      dtype=object)

<IPython.core.display.Javascript object>

In [32]:
df_results_cm = (
    df_pre_train[df_pre_train["Features"].eq("Chemical + Mineralogical")]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [33]:
df_results_cm["Features"].unique()

array(['Chemical + Mineralogical'], dtype=object)

<IPython.core.display.Javascript object>

In [34]:
df_results_cm.shape

(73, 26)

<IPython.core.display.Javascript object>

# Global Analysis (pre train results)

## ECICS - 206

### Variable Grouping: CM

#### Analysis of the best models identified in the Local Analysis

**Best models for variable groupping CM were from:**

*BTiSS:*
    Dominance analysis: Plant B
    SCPM:Plant B

*TiSS:*
    Dominance analysis: Plant B
    SCPM:Plant B

### Plant B

#### Hold Out

In [35]:
df_results_cm = (
    df_pre_train[df_pre_train["Features"].eq("Chemical + Mineralogical")]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [36]:
df_results_cm["Features"].unique()

array(['Chemical + Mineralogical'], dtype=object)

<IPython.core.display.Javascript object>

In [37]:
df_results_cm.shape

(73, 26)

<IPython.core.display.Javascript object>

In [38]:
df_results_cm = df_results_cm[df_results_cm["Plant"].eq("B")].reset_index(drop=True)

<IPython.core.display.Javascript object>

In [39]:
df_results_cm["Cross Validation"].unique()

array(['Out of time'], dtype=object)

<IPython.core.display.Javascript object>

In [40]:
df_results_cm_ho = (
    df_results_cm[df_results_cm["Cross Validation"].eq("Out of time")]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [41]:
df_results_cm_ho["Cross Validation"].unique()

array(['Out of time'], dtype=object)

<IPython.core.display.Javascript object>

In [42]:
df_results_cm_ho.shape

(73, 26)

<IPython.core.display.Javascript object>

###### SCPM computation

In [43]:
df_results_cm_ho = compute_scpm(df_results_cm_ho)

<IPython.core.display.Javascript object>

##### Dominance Analysis

In [44]:
dominance_dict = make_dominance_analysis(df_results_cm_ho)
dominance_matrix_cm_ho = dominance_dict["dominance_matrix"]
dominance_graph_cm_ho = dominance_dict["dominance_graph"]
df_sorted_count = dominance_dict["df_sorted_count"]
df_sorted_topo = dominance_dict["df_sorted_topo"]

<IPython.core.display.Javascript object>

In [45]:
dominance_matrix_cm_ho.shape, len(dominance_graph_cm_ho)

((73, 73), 73)

<IPython.core.display.Javascript object>

In [46]:
df_sorted_topo["Classification"].value_counts()

Classification
intermediate_model    71
dominant_model         1
non_dominant_model     1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [47]:
df_sorted_topo.head(5)

,Category,Company,Plant,Features,Data Shape,Timesteps,Model,Model Params,Scaler,Scaler Params,...,MAPE_mean,R2_mean,plant,Features_bkp,Model_bkp,Model_bkp_2,SCPM,Dominated_Count,Classification,Dominates_Count
24,Global Model,204,B,Chemical + Mineralogical,"(64121, 4)",14.0,LSTM12,NaN,Standard Scaler,NaN,...,4.471390,0.819314,b,Chemical,LSTM12,LSTM,-0.857461,0,dominant_model,72
14,Global Model,204,B,Chemical + Mineralogical,"(64121, 4)",7.0,LSTM2,NaN,Standard Scaler,NaN,...,4.488177,0.818546,b,Chemical,LSTM2,LSTM,-0.852573,1,intermediate_model,71
29,Global Model,206,B,Chemical + Mineralogical,"(64121, 4)",7.0,Bi-LSTM2,NaN,Standard Scaler,NaN,...,4.519476,0.810398,b,Chemical,Bi-LSTM2,Bi-LSTM,-0.825700,2,intermediate_model,69
30,Global Model,206,B,Chemical + Mineralogical,"(64121, 4)",14.0,Bi-LSTM3,NaN,Standard Scaler,NaN,...,4.512617,0.812702,b,Chemical,Bi-LSTM3,Bi-LSTM,-0.830068,2,intermediate_model,68
41,Global Model,206,B,Chemical + Mineralogical,"(64121, 4)",7.0,Bi-LSTM14,NaN,Standard Scaler,NaN,...,4.535540,0.804382,b,Chemical,Bi-LSTM14,Bi-LSTM,-0.811376,3,intermediate_model,66


<IPython.core.display.Javascript object>

In [48]:
df_sorted_topo_cols = df_sorted_topo[
    [
        "Model",
        # "Model_bkp",
        # "Model_bkp_2",
        "Timesteps",
        "RMSE_mean",
        "MAE_mean",
        "MAPE_mean",
        "R2_mean",
        "SCPM",
        "Dominated_Count",
        "Classification",
        "Dominates_Count",
        "Plant",
    ]
].round(2)

<IPython.core.display.Javascript object>

##### Top dominant models

In [49]:
# Top 4
df_sorted_topo_cols.sort_values(by="Dominates_Count", ascending=False).iloc[0:3]

,Model,Timesteps,RMSE_mean,MAE_mean,MAPE_mean,R2_mean,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
24,LSTM12,14.0,2.56,1.92,4.47,0.82,-0.86,0,dominant_model,72,B
14,LSTM2,7.0,2.56,1.93,4.49,0.82,-0.85,1,intermediate_model,71,B
29,Bi-LSTM2,7.0,2.62,1.96,4.52,0.81,-0.83,2,intermediate_model,69,B


<IPython.core.display.Javascript object>

##### Top intermediate models

In [50]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("intermediate_model")
].sort_values(by="Dominates_Count", ascending=False).iloc[:2]

,Model,Timesteps,RMSE_mean,MAE_mean,MAPE_mean,R2_mean,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
14,LSTM2,7.0,2.56,1.93,4.49,0.82,-0.85,1,intermediate_model,71,B
29,Bi-LSTM2,7.0,2.62,1.96,4.52,0.81,-0.83,2,intermediate_model,69,B


<IPython.core.display.Javascript object>

##### Top non dominant models

In [51]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("non_dominant_model")
].sort_values(by="Dominated_Count", ascending=True).iloc[:2]

,Model,Timesteps,RMSE_mean,MAE_mean,MAPE_mean,R2_mean,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
39,Bi-LSTM12,14.0,36.27,35.77,82.27,-35.29,33.85,72,non_dominant_model,0,B


<IPython.core.display.Javascript object>

In [52]:
df_sorted_topo_models_cols = df_sorted_topo[
    [
        # "Model",
        # "Model_bkp",
        "Model_bkp_2",
        "Timesteps",
        "RMSE_mean",
        "MAE_mean",
        "MAPE_mean",
        "R2_mean",
        "SCPM",
        "Dominated_Count",
        "Classification",
        "Dominates_Count",
        "Plant",
    ]
]  # .round(2)

<IPython.core.display.Javascript object>

In [53]:
df_sorted_topo["Model_bkp_2"].value_counts()

Model_bkp_2
LSTM           15
Bi-LSTM        15
Conv1D         15
Transformer    15
MLP            13
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [54]:
df_sorted_topo_models_grouped = (
    df_sorted_topo_models_cols.groupby("Model_bkp_2").describe().T
)

<IPython.core.display.Javascript object>

In [55]:
# pd.set_option("display.max_rows", None)
# df_sorted_topo_models_grouped

<IPython.core.display.Javascript object>

In [56]:
df_sorted_topo_models_cols = df_sorted_topo[
    [
        "Model_bkp_2",
        "Dominated_Count",
        "Dominates_Count",
        "Classification",
    ]
]

<IPython.core.display.Javascript object>

In [57]:
df_sorted_topo_models_cols.groupby("Model_bkp_2").describe().round(2)

Dominated_Count                                              \
                      count   mean    std   min   25%   50%   75%   max   
Model_bkp_2                                                               
Bi-LSTM                15.0  29.60  25.20   2.0   9.0  20.0  51.5  72.0   
Conv1D                 15.0  24.07  11.67   4.0  13.5  23.0  34.5  41.0   
LSTM                   15.0  22.80  22.87   0.0   6.5  11.0  45.0  65.0   
MLP                    13.0  43.08   9.33  31.0  35.0  40.0  51.0  58.0   
Transformer            15.0  46.07  17.34  29.0  32.5  36.0  67.0  71.0   

            Dominates_Count                                              
                      count   mean    std   min   25%   50%   75%   max  
Model_bkp_2                                                              
Bi-LSTM                15.0  38.00  25.20   0.0  12.5  46.0  57.5  69.0  
Conv1D                 15.0  41.27  14.15  20.0  26.5  46.0  50.0  65.0  
LSTM                   15.0  45.07  25.13   6.0  17.5  55.0  63.5  72.0  
MLP                    13.0  18.92   8.94   9.0  10.0  20.0  26.0  34.0  
Transformer            15.0  19.13  15.55   1.0   4.0  20.0  31.5  43.0

<IPython.core.display.Javascript object>

In [58]:
df_sorted_topo_models_cols.groupby("Model_bkp_2")["Classification"].describe()

,count,unique,top,freq
Model_bkp_2,,,,
Bi-LSTM,15,2,intermediate_model,14
Conv1D,15,1,intermediate_model,15
LSTM,15,2,intermediate_model,14
MLP,13,1,intermediate_model,13
Transformer,15,1,intermediate_model,15


<IPython.core.display.Javascript object>

In [59]:
df_sorted_topo_models_cols.groupby("Model_bkp_2")[["Classification"]].value_counts()

Model_bkp_2  Classification    
Bi-LSTM      intermediate_model    14
             non_dominant_model     1
Conv1D       intermediate_model    15
LSTM         intermediate_model    14
             dominant_model         1
MLP          intermediate_model    13
Transformer  intermediate_model    15
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [60]:
(
    df_sorted_topo_models_cols.groupby("Model_bkp_2")[["Classification"]]
    .value_counts(normalize=True)
    .to_frame()
    * 100
).round(2)

proportion
Model_bkp_2 Classification                
Bi-LSTM     intermediate_model       93.33
            non_dominant_model        6.67
Conv1D      intermediate_model      100.00
LSTM        intermediate_model       93.33
            dominant_model            6.67
MLP         intermediate_model      100.00
Transformer intermediate_model      100.00

<IPython.core.display.Javascript object>

In [61]:
df_sorted_topo_models_cols

,Model_bkp_2,Dominated_Count,Dominates_Count,Classification
24,LSTM,0,72,dominant_model
14,LSTM,1,71,intermediate_model
29,Bi-LSTM,2,69,intermediate_model
30,Bi-LSTM,2,68,intermediate_model
41,Bi-LSTM,3,66,intermediate_model
...,...,...,...,...
55,Transformer,67,4,intermediate_model
46,Transformer,69,3,intermediate_model
43,Transformer,70,2,intermediate_model
52,Transformer,71,1,intermediate_model


<IPython.core.display.Javascript object>

In [62]:
summary_stats = df_sorted_topo_models_cols.groupby("Model_bkp_2").agg(
    Total_Models=("Model_bkp_2", "count"),
    Mean_Dominance_Count=("Dominates_Count", "mean"),
    Median_Dominance_Count=("Dominates_Count", "median"),
    Total_Dominated=("Dominated_Count", "sum"),
    Total_Dominating=("Dominates_Count", "sum"),
)

total_models = summary_stats[
    "Total_Models"
].sum()  # Get the total number of models across all categories
summary_stats["Dominance Proportion"] = summary_stats["Total_Dominating"] / total_models

<IPython.core.display.Javascript object>

In [63]:
summary_stats

,Total_Models,Mean_Dominance_Count,Median_Dominance_Count,Total_Dominated,Total_Dominating,Dominance Proportion
Model_bkp_2,,,,,,
Bi-LSTM,15,38.000000,46.0,444,570,7.808219
Conv1D,15,41.266667,46.0,361,619,8.479452
LSTM,15,45.066667,55.0,342,676,9.260274
MLP,13,18.923077,20.0,560,246,3.369863
Transformer,15,19.133333,20.0,691,287,3.931507


<IPython.core.display.Javascript object>

In [64]:
df_sorted_topo_models_grouped.T

Timesteps                                                 \
                count      mean       std  min  25%  50%   75%   max   
Model_bkp_2                                                            
Bi-LSTM          15.0  7.333333  5.498918  1.0  1.0  7.0  14.0  14.0   
Conv1D           15.0  7.333333  5.498918  1.0  1.0  7.0  14.0  14.0   
LSTM             15.0  7.333333  5.498918  1.0  1.0  7.0  14.0  14.0   
MLP               0.0       NaN       NaN  NaN  NaN  NaN   NaN   NaN   
Transformer      15.0  7.333333  5.498918  1.0  1.0  7.0  14.0  14.0   

            RMSE_mean            ... Dominated_Count       Dominates_Count  \
                count      mean  ...             75%   max           count   
Model_bkp_2                      ...                                         
Bi-LSTM          15.0  5.208711  ...            51.5  72.0            15.0   
Conv1D           15.0  2.941359  ...            34.5  41.0            15.0   
LSTM             15.0  2.928592  ...            45.0  65.0            15.0   
MLP              13.0  3.317524  ...            51.0  58.0            13.0   
Transformer      15.0  3.425188  ...            67.0  71.0            15.0   

                                                                 
                  mean        std   min   25%   50%   75%   max  
Model_bkp_2                                                      
Bi-LSTM      38.000000  25.202041   0.0  12.5  46.0  57.5  69.0  
Conv1D       41.266667  14.154589  20.0  26.5  46.0  50.0  65.0  
LSTM         45.066667  25.126726   6.0  17.5  55.0  63.5  72.0  
MLP          18.923077   8.939254   9.0  10.0  20.0  26.0  34.0  
Transformer  19.133333  15.546551   1.0   4.0  20.0  31.5  43.0  

[5 rows x 64 columns]

<IPython.core.display.Javascript object>

### SCPM Analysis

In [65]:
cols = [
    "Model",
    # "Model_bkp",
    "Model_bkp_2",
    "Timesteps",
    "RMSE_mean",
    "MAE_mean",
    "MAPE_mean",
    "R2_mean",
    "SCPM",
    "Dominated_Count",
    "Dominates_Count",
    "Classification",
    "Plant",
]

<IPython.core.display.Javascript object>

In [66]:
df_sorted_scpm = df_sorted_topo.sort_values(by="SCPM").copy()

<IPython.core.display.Javascript object>

In [67]:
# (df_sorted_topo[cols].reset_index() == df_sorted_scpm[cols].reset_index()).all(axis=1)
df_sorted_scpm["topo_scmp_order_eq"] = df_sorted_topo.index == df_sorted_scpm.index

<IPython.core.display.Javascript object>

In [68]:
# df_sorted_topo[cols].round(2).head(5)

<IPython.core.display.Javascript object>

In [69]:
df_sorted_scpm[cols + ["topo_scmp_order_eq"]].round(2).head(7)

,Model,Model_bkp_2,Timesteps,RMSE_mean,MAE_mean,MAPE_mean,R2_mean,SCPM,Dominated_Count,Dominates_Count,Classification,Plant,topo_scmp_order_eq
24,LSTM12,LSTM,14.0,2.56,1.92,4.47,0.82,-0.86,0,72,dominant_model,B,True
14,LSTM2,LSTM,7.0,2.56,1.93,4.49,0.82,-0.85,1,71,intermediate_model,B,True
30,Bi-LSTM3,Bi-LSTM,14.0,2.61,1.96,4.51,0.81,-0.83,2,68,intermediate_model,B,False
29,Bi-LSTM2,Bi-LSTM,7.0,2.62,1.96,4.52,0.81,-0.83,2,69,intermediate_model,B,False
21,LSTM9,LSTM,14.0,2.65,1.97,4.54,0.81,-0.81,4,66,intermediate_model,B,False
41,Bi-LSTM14,Bi-LSTM,7.0,2.66,1.96,4.54,0.80,-0.81,3,66,intermediate_model,B,False
71,Conv1D_14,Conv1D,7.0,2.63,1.98,4.64,0.81,-0.80,4,65,intermediate_model,B,True


<IPython.core.display.Javascript object>

In [70]:
df_sorted_scpm[cols + ["topo_scmp_order_eq"]].head(1)

,Model,Model_bkp_2,Timesteps,RMSE_mean,MAE_mean,MAPE_mean,R2_mean,SCPM,Dominated_Count,Dominates_Count,Classification,Plant,topo_scmp_order_eq
24,LSTM12,LSTM,14.0,2.559414,1.922963,4.47139,0.819314,-0.857461,0,72,dominant_model,B,True


<IPython.core.display.Javascript object>

### Variable Grouping: CM-P

#### Analysis of the best models identified in the Local Analysis

**Best models for variable groupping CM-P were from:**

*BTiSS:*
    Dominance analysis: Plant B
    SCPM:Plant B

*TiSS:*
    Dominance analysis: Plant B
    SCPM:Plant B

In [71]:
df_pre_train["Features"].unique()

array(['Chemical + Mineralogical + Physical - Early CS',
       'Chemical + Mineralogical + Physical', 'Chemical + Mineralogical'],
      dtype=object)

<IPython.core.display.Javascript object>

In [72]:
df_results_cm_p = (
    df_pre_train[
        df_pre_train["Features"].eq("Chemical + Mineralogical + Physical - Early CS")
    ]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

### Plant B

In [73]:
df_results_cm_p_am = df_results_cm_p[df_results_cm_p["Plant"].eq("B")].reset_index(
    drop=True
)

<IPython.core.display.Javascript object>

In [74]:
df_results_cm_p_am.shape

(73, 26)

<IPython.core.display.Javascript object>

#### Hold Out

In [75]:
df_results_cm_p_am["Cross Validation"].unique()

array(['Out of time'], dtype=object)

<IPython.core.display.Javascript object>

In [76]:
df_results_cm_p_am_ho = (
    df_results_cm_p_am[df_results_cm_p_am["Cross Validation"].eq("Out of time")]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [77]:
df_results_cm_p_am_ho["Cross Validation"].unique()

array(['Out of time'], dtype=object)

<IPython.core.display.Javascript object>

In [78]:
df_results_cm_p_am_ho.shape

(73, 26)

<IPython.core.display.Javascript object>

###### SCPM computation

In [79]:
df_results_cm_p_am_ho = compute_scpm(df_results_cm_p_am_ho)

<IPython.core.display.Javascript object>

##### Dominance Analysis

In [80]:
dominance_dict = make_dominance_analysis(df_results_cm_p_am_ho)
dominance_matrix_cm_p_am_ho = dominance_dict["dominance_matrix"]
dominance_graph_cm_p_am_ho = dominance_dict["dominance_graph"]
df_sorted_count = dominance_dict["df_sorted_count"]
df_sorted_topo = dominance_dict["df_sorted_topo"]

<IPython.core.display.Javascript object>

In [81]:
dominance_matrix_cm_p_am_ho.shape, len(dominance_graph_cm_p_am_ho)

((73, 73), 73)

<IPython.core.display.Javascript object>

In [82]:
df_sorted_topo["Classification"].value_counts()

Classification
intermediate_model    69
dominant_model         3
non_dominant_model     1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [83]:
df_sorted_topo.head(5)

,Category,Company,Plant,Features,Data Shape,Timesteps,Model,Model Params,Scaler,Scaler Params,...,MAPE_mean,R2_mean,plant,Features_bkp,Model_bkp,Model_bkp_2,SCPM,Dominated_Count,Classification,Dominates_Count
65,Global Model,206,B,Chemical + Mineralogical + Physical - Early CS,"(64121, 9)",NaN,MLP_6,NaN,Standard Scaler,NaN,...,4.041626,0.856446,b,Chemical + Properties CS Less,MLP_6,MLP,-1.368537,0,dominant_model,70
69,Global Model,206,B,Chemical + Mineralogical + Physical - Early CS,"(64121, 9)",NaN,MLP_10,NaN,Standard Scaler,NaN,...,3.939018,0.850730,b,Chemical + Properties CS Less,MLP_10,MLP,-1.376359,0,dominant_model,70
72,Global Model,206,B,Chemical + Mineralogical + Physical - Early CS,"(64121, 9)",NaN,MLP_13,NaN,Standard Scaler,NaN,...,4.020180,0.852655,b,Chemical + Properties CS Less,MLP_13,MLP,-1.365847,0,dominant_model,70
71,Global Model,206,B,Chemical + Mineralogical + Physical - Early CS,"(64121, 9)",NaN,MLP_12,NaN,Standard Scaler,NaN,...,4.101876,0.849240,b,Chemical + Properties CS Less,MLP_12,MLP,-1.337078,3,intermediate_model,69
66,Global Model,206,B,Chemical + Mineralogical + Physical - Early CS,"(64121, 9)",NaN,MLP_7,NaN,Standard Scaler,NaN,...,4.109957,0.847165,b,Chemical + Properties CS Less,MLP_7,MLP,-1.328854,4,intermediate_model,67


<IPython.core.display.Javascript object>

In [84]:
df_sorted_topo_cols = df_sorted_topo[
    [
        "Model",
        # "Model_bkp",
        # "Model_bkp_2",
        "Timesteps",
        "RMSE_mean",
        "MAE_mean",
        "MAPE_mean",
        "R2_mean",
        "SCPM",
        "Dominated_Count",
        "Classification",
        "Dominates_Count",
        "Plant",
    ]
].round(2)

<IPython.core.display.Javascript object>

##### Top dominant models

In [85]:
# Top 4
df_sorted_topo_cols.sort_values(by="Dominates_Count", ascending=False).iloc[0:3]

,Model,Timesteps,RMSE_mean,MAE_mean,MAPE_mean,R2_mean,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
65,MLP_6,NaN,2.28,1.72,4.04,0.86,-1.37,0,dominant_model,70,B
69,MLP_10,NaN,2.33,1.68,3.94,0.85,-1.38,0,dominant_model,70,B
72,MLP_13,NaN,2.31,1.70,4.02,0.85,-1.37,0,dominant_model,70,B


<IPython.core.display.Javascript object>

##### Top intermediate models

In [86]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("intermediate_model")
].sort_values(by="Dominates_Count", ascending=False).iloc[:2]

,Model,Timesteps,RMSE_mean,MAE_mean,MAPE_mean,R2_mean,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
71,MLP_12,NaN,2.34,1.75,4.10,0.85,-1.34,3,intermediate_model,69,B
70,MLP_11,NaN,2.35,1.75,4.13,0.85,-1.33,4,intermediate_model,67,B


<IPython.core.display.Javascript object>

##### Top non dominant models

In [87]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("non_dominant_model")
].sort_values(by="Dominated_Count", ascending=True).iloc[:2]

,Model,Timesteps,RMSE_mean,MAE_mean,MAPE_mean,R2_mean,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
32,Bi-LSTM3,14.0,33.06,32.51,74.59,-29.15,32.14,72,non_dominant_model,0,B


<IPython.core.display.Javascript object>

In [88]:
df_sorted_topo_models_cols = df_sorted_topo[
    [
        # "Model",
        # "Model_bkp",
        "Model_bkp_2",
        "Timesteps",
        "RMSE_mean",
        "MAE_mean",
        "MAPE_mean",
        "R2_mean",
        "SCPM",
        "Dominated_Count",
        "Classification",
        "Dominates_Count",
        "Plant",
    ]
]  # .round(2)

<IPython.core.display.Javascript object>

In [89]:
df_sorted_topo["Model_bkp_2"].value_counts()

Model_bkp_2
Conv1D         15
Bi-LSTM        15
LSTM           15
Transformer    15
MLP            13
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [90]:
df_sorted_topo_models_grouped = (
    df_sorted_topo_models_cols.groupby("Model_bkp_2").describe().T
)

<IPython.core.display.Javascript object>

In [91]:
# pd.set_option("display.max_rows", None)
# df_sorted_topo_models_grouped

<IPython.core.display.Javascript object>

In [92]:
df_sorted_topo_models_cols = df_sorted_topo[
    [
        "Model_bkp_2",
        "Dominated_Count",
        "Dominates_Count",
        "Classification",
    ]
]

<IPython.core.display.Javascript object>

In [93]:
df_sorted_topo_models_cols.groupby("Model_bkp_2").describe().round(2)

Dominated_Count                                              \
                      count   mean    std   min   25%   50%   75%   max   
Model_bkp_2                                                               
Bi-LSTM                15.0  48.80  17.63  14.0  36.5  53.0  64.5  72.0   
Conv1D                 15.0  19.73   6.09  11.0  15.0  20.0  23.0  34.0   
LSTM                   15.0  49.60  15.45  26.0  35.5  58.0  62.0  67.0   
MLP                    13.0   5.23   3.83   0.0   3.0   6.0   7.0  11.0   
Transformer            15.0  46.67  10.57  26.0  42.0  47.0  51.5  69.0   

            Dominates_Count                                              
                      count   mean    std   min   25%   50%   75%   max  
Model_bkp_2                                                              
Bi-LSTM                15.0  20.47  15.84   0.0   6.5  16.0  33.0  50.0  
Conv1D                 15.0  48.93   6.44  34.0  47.5  48.0  53.5  59.0  
LSTM                   15.0  20.47  14.43   4.0   9.5  13.0  34.0  42.0  
MLP                    13.0  65.08   4.07  59.0  63.0  65.0  69.0  70.0  
Transformer            15.0  23.07   9.64   2.0  20.5  22.0  26.0  40.0

<IPython.core.display.Javascript object>

In [94]:
df_sorted_topo_models_cols.groupby("Model_bkp_2")["Classification"].describe()

,count,unique,top,freq
Model_bkp_2,,,,
Bi-LSTM,15,2,intermediate_model,14
Conv1D,15,1,intermediate_model,15
LSTM,15,1,intermediate_model,15
MLP,13,2,intermediate_model,10
Transformer,15,1,intermediate_model,15


<IPython.core.display.Javascript object>

In [95]:
df_sorted_topo_models_cols.groupby("Model_bkp_2")[["Classification"]].value_counts()

Model_bkp_2  Classification    
Bi-LSTM      intermediate_model    14
             non_dominant_model     1
Conv1D       intermediate_model    15
LSTM         intermediate_model    15
MLP          intermediate_model    10
             dominant_model         3
Transformer  intermediate_model    15
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [96]:
(
    df_sorted_topo_models_cols.groupby("Model_bkp_2")[["Classification"]]
    .value_counts(normalize=True)
    .to_frame()
    * 100
).round(2)

proportion
Model_bkp_2 Classification                
Bi-LSTM     intermediate_model       93.33
            non_dominant_model        6.67
Conv1D      intermediate_model      100.00
LSTM        intermediate_model      100.00
MLP         intermediate_model       76.92
            dominant_model           23.08
Transformer intermediate_model      100.00

<IPython.core.display.Javascript object>

In [97]:
df_sorted_topo_models_cols

,Model_bkp_2,Dominated_Count,Dominates_Count,Classification
65,MLP,0,70,dominant_model
69,MLP,0,70,dominant_model
72,MLP,0,70,dominant_model
71,MLP,3,69,intermediate_model
66,MLP,4,67,intermediate_model
...,...,...,...,...
13,LSTM,67,4,intermediate_model
35,Bi-LSTM,69,2,intermediate_model
57,Transformer,69,2,intermediate_model
38,Bi-LSTM,71,1,intermediate_model


<IPython.core.display.Javascript object>

In [98]:
summary_stats = df_sorted_topo_models_cols.groupby("Model_bkp_2").agg(
    Total_Models=("Model_bkp_2", "count"),
    Mean_Dominance_Count=("Dominates_Count", "mean"),
    Median_Dominance_Count=("Dominates_Count", "median"),
    Total_Dominated=("Dominated_Count", "sum"),
    Total_Dominating=("Dominates_Count", "sum"),
)

total_models = summary_stats[
    "Total_Models"
].sum()  # Get the total number of models across all categories
summary_stats["Dominance Proportion"] = summary_stats["Total_Dominating"] / total_models

<IPython.core.display.Javascript object>

In [99]:
summary_stats

,Total_Models,Mean_Dominance_Count,Median_Dominance_Count,Total_Dominated,Total_Dominating,Dominance Proportion
Model_bkp_2,,,,,,
Bi-LSTM,15,20.466667,16.0,732,307,4.205479
Conv1D,15,48.933333,48.0,296,734,10.054795
LSTM,15,20.466667,13.0,744,307,4.205479
MLP,13,65.076923,65.0,68,846,11.589041
Transformer,15,23.066667,22.0,700,346,4.739726


<IPython.core.display.Javascript object>

In [100]:
df_sorted_topo_models_grouped.T

Timesteps                                                 \
                count      mean       std  min  25%  50%   75%   max   
Model_bkp_2                                                            
Bi-LSTM          15.0  7.333333  5.498918  1.0  1.0  7.0  14.0  14.0   
Conv1D           15.0  7.333333  5.498918  1.0  1.0  7.0  14.0  14.0   
LSTM             15.0  7.333333  5.498918  1.0  1.0  7.0  14.0  14.0   
MLP               0.0       NaN       NaN  NaN  NaN  NaN   NaN   NaN   
Transformer      15.0  7.333333  5.498918  1.0  1.0  7.0  14.0  14.0   

            RMSE_mean            ... Dominated_Count       Dominates_Count  \
                count      mean  ...             75%   max           count   
Model_bkp_2                      ...                                         
Bi-LSTM          15.0  6.299990  ...            64.5  72.0            15.0   
Conv1D           15.0  2.658165  ...            23.0  34.0            15.0   
LSTM             15.0  4.066082  ...            62.0  67.0            15.0   
MLP              13.0  2.398722  ...             7.0  11.0            13.0   
Transformer      15.0  3.612745  ...            51.5  69.0            15.0   

                                                                 
                  mean        std   min   25%   50%   75%   max  
Model_bkp_2                                                      
Bi-LSTM      20.466667  15.837869   0.0   6.5  16.0  33.0  50.0  
Conv1D       48.933333   6.441680  34.0  47.5  48.0  53.5  59.0  
LSTM         20.466667  14.426497   4.0   9.5  13.0  34.0  42.0  
MLP          65.076923   4.071477  59.0  63.0  65.0  69.0  70.0  
Transformer  23.066667   9.639700   2.0  20.5  22.0  26.0  40.0  

[5 rows x 64 columns]

<IPython.core.display.Javascript object>

### SCPM Analysis

In [101]:
cols = [
    "Model",
    # "Model_bkp",
    "Model_bkp_2",
    "Timesteps",
    "RMSE_mean",
    "MAE_mean",
    "MAPE_mean",
    "R2_mean",
    "SCPM",
    "Dominated_Count",
    "Dominates_Count",
    "Classification",
    "Plant",
]

<IPython.core.display.Javascript object>

In [102]:
df_sorted_scpm = df_sorted_topo.sort_values(by="SCPM").copy()

<IPython.core.display.Javascript object>

In [103]:
# (df_sorted_topo[cols].reset_index() == df_sorted_scpm[cols].reset_index()).all(axis=1)
df_sorted_scpm["topo_scmp_order_eq"] = df_sorted_topo.index == df_sorted_scpm.index

<IPython.core.display.Javascript object>

In [104]:
# df_sorted_topo[cols].round(2).head(5)

<IPython.core.display.Javascript object>

In [105]:
df_sorted_scpm[cols + ["topo_scmp_order_eq"]].round(2).head(7)

,Model,Model_bkp_2,Timesteps,RMSE_mean,MAE_mean,MAPE_mean,R2_mean,SCPM,Dominated_Count,Dominates_Count,Classification,Plant,topo_scmp_order_eq
69,MLP_10,MLP,NaN,2.33,1.68,3.94,0.85,-1.38,0,70,dominant_model,B,False
65,MLP_6,MLP,NaN,2.28,1.72,4.04,0.86,-1.37,0,70,dominant_model,B,False
72,MLP_13,MLP,NaN,2.31,1.70,4.02,0.85,-1.37,0,70,dominant_model,B,True
71,MLP_12,MLP,NaN,2.34,1.75,4.10,0.85,-1.34,3,69,intermediate_model,B,True
66,MLP_7,MLP,NaN,2.35,1.75,4.11,0.85,-1.33,4,67,intermediate_model,B,True
70,MLP_11,MLP,NaN,2.35,1.75,4.13,0.85,-1.33,4,67,intermediate_model,B,True
64,MLP_5,MLP,NaN,2.42,1.78,4.19,0.84,-1.29,6,65,intermediate_model,B,False


<IPython.core.display.Javascript object>

### Variable Grouping: CM-P-CS

#### Analysis of the best models identified in the Local Analysis

**Best models for variable groupping CM-P-CS were from:**

*BTiSS:*
    Dominance analysis: Plant B
    SCPM:Plant B

*TiSS:*
    Dominance analysis: Plant B
    SCPM:Plant B

In [106]:
df_pre_train["Features"].unique()

array(['Chemical + Mineralogical + Physical - Early CS',
       'Chemical + Mineralogical + Physical', 'Chemical + Mineralogical'],
      dtype=object)

<IPython.core.display.Javascript object>

### Plant B

In [107]:
df_results_cm_p_cs = (
    df_pre_train[df_pre_train["Features"].eq("Chemical + Mineralogical + Physical")]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [108]:
df_results_cm_p_cs_s = df_results_cm_p_cs[
    df_results_cm_p_cs["Plant"].eq("B")
].reset_index(drop=True)

<IPython.core.display.Javascript object>

In [109]:
df_results_cm_p_cs_s.shape

(73, 26)

<IPython.core.display.Javascript object>

#### Hold Out

In [110]:
df_results_cm_p_cs_s["Cross Validation"].unique()

array(['Out of time'], dtype=object)

<IPython.core.display.Javascript object>

In [111]:
df_results_cm_p_cs_s_ho = (
    df_results_cm_p_cs_s[df_results_cm_p_cs_s["Cross Validation"].eq("Out of time")]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [112]:
df_results_cm_p_cs_s_ho["Cross Validation"].unique()

array(['Out of time'], dtype=object)

<IPython.core.display.Javascript object>

In [113]:
df_results_cm_p_cs_s_ho.shape

(73, 26)

<IPython.core.display.Javascript object>

###### SCPM computation

In [114]:
df_results_cm_p_cs_s_ho = compute_scpm(df_results_cm_p_cs_s_ho)

<IPython.core.display.Javascript object>

##### Dominance Analysis

In [115]:
dominance_dict = make_dominance_analysis(df_results_cm_p_cs_s_ho)
dominance_matrix_cm_p_cs_s_ho = dominance_dict["dominance_matrix"]
dominance_graph_cm_p_cs_s_ho = dominance_dict["dominance_graph"]
df_sorted_count = dominance_dict["df_sorted_count"]
df_sorted_topo = dominance_dict["df_sorted_topo"]

<IPython.core.display.Javascript object>

In [116]:
dominance_matrix_cm_p_cs_s_ho.shape, len(dominance_graph_cm_p_cs_s_ho)

((73, 73), 73)

<IPython.core.display.Javascript object>

In [117]:
df_sorted_topo["Classification"].value_counts()

Classification
intermediate_model    71
dominant_model         1
non_dominant_model     1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [118]:
df_sorted_topo.head(5)

,Category,Company,Plant,Features,Data Shape,Timesteps,Model,Model Params,Scaler,Scaler Params,...,MAPE_mean,R2_mean,plant,Features_bkp,Model_bkp,Model_bkp_2,SCPM,Dominated_Count,Classification,Dominates_Count
25,Global Model,206,B,Chemical + Mineralogical + Physical,"(64121, 12)",NaN,MLP_11,NaN,Standard Scaler,NaN,...,2.687074,0.934304,b,Chemical + Physical,MLP_11,MLP,-1.448854,0,dominant_model,72
20,Global Model,206,B,Chemical + Mineralogical + Physical,"(64121, 12)",NaN,MLP_6,NaN,Standard Scaler,NaN,...,2.702276,0.931831,b,Chemical + Physical,MLP_6,MLP,-1.436006,1,intermediate_model,71
24,Global Model,206,B,Chemical + Mineralogical + Physical,"(64121, 12)",NaN,MLP_10,NaN,Standard Scaler,NaN,...,2.721203,0.930872,b,Chemical + Physical,MLP_10,MLP,-1.430491,2,intermediate_model,70
19,Global Model,206,B,Chemical + Mineralogical + Physical,"(64121, 12)",NaN,MLP_5,NaN,Standard Scaler,NaN,...,2.801666,0.928945,b,Chemical + Physical,MLP_5,MLP,-1.407567,3,intermediate_model,67
21,Global Model,206,B,Chemical + Mineralogical + Physical,"(64121, 12)",NaN,MLP_7,NaN,Standard Scaler,NaN,...,2.804222,0.930192,b,Chemical + Physical,MLP_7,MLP,-1.412525,3,intermediate_model,68


<IPython.core.display.Javascript object>

In [119]:
df_sorted_topo_cols = df_sorted_topo[
    [
        "Model",
        # "Model_bkp",
        # "Model_bkp_2",
        "Timesteps",
        "RMSE_mean",
        "MAE_mean",
        "MAPE_mean",
        "R2_mean",
        "SCPM",
        "Dominated_Count",
        "Classification",
        "Dominates_Count",
        "Plant",
    ]
].round(2)

<IPython.core.display.Javascript object>

##### Top dominant models

In [120]:
# Top 4
df_sorted_topo_cols.sort_values(by="Dominates_Count", ascending=False).iloc[0:3]

,Model,Timesteps,RMSE_mean,MAE_mean,MAPE_mean,R2_mean,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
25,MLP_11,NaN,1.54,1.15,2.69,0.93,-1.45,0,dominant_model,72,B
20,MLP_6,NaN,1.57,1.17,2.70,0.93,-1.44,1,intermediate_model,71,B
24,MLP_10,NaN,1.58,1.17,2.72,0.93,-1.43,2,intermediate_model,70,B


<IPython.core.display.Javascript object>

##### Top intermediate models

In [121]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("intermediate_model")
].sort_values(by="Dominates_Count", ascending=False).iloc[:2]

,Model,Timesteps,RMSE_mean,MAE_mean,MAPE_mean,R2_mean,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
20,MLP_6,NaN,1.57,1.17,2.70,0.93,-1.44,1,intermediate_model,71,B
24,MLP_10,NaN,1.58,1.17,2.72,0.93,-1.43,2,intermediate_model,70,B


<IPython.core.display.Javascript object>

##### Top non dominant models

In [122]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("non_dominant_model")
].sort_values(by="Dominated_Count", ascending=True).iloc[:2]

,Model,Timesteps,RMSE_mean,MAE_mean,MAPE_mean,R2_mean,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
54,Bi-LSTM12,14.0,28.06,27.4,62.55,-20.71,25.3,72,non_dominant_model,0,B


<IPython.core.display.Javascript object>

In [123]:
df_sorted_topo_models_cols = df_sorted_topo[
    [
        # "Model",
        # "Model_bkp",
        "Model_bkp_2",
        "Timesteps",
        "RMSE_mean",
        "MAE_mean",
        "MAPE_mean",
        "R2_mean",
        "SCPM",
        "Dominated_Count",
        "Classification",
        "Dominates_Count",
        "Plant",
    ]
]  # .round(2)

<IPython.core.display.Javascript object>

In [124]:
df_sorted_topo["Model_bkp_2"].value_counts()

Model_bkp_2
Conv1D         15
Bi-LSTM        15
LSTM           15
Transformer    15
MLP            13
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [125]:
df_sorted_topo_models_grouped = (
    df_sorted_topo_models_cols.groupby("Model_bkp_2").describe().T
)

<IPython.core.display.Javascript object>

In [126]:
# pd.set_option("display.max_rows", None)
# df_sorted_topo_models_grouped

<IPython.core.display.Javascript object>

In [127]:
df_sorted_topo_models_cols = df_sorted_topo[
    [
        "Model_bkp_2",
        "Dominated_Count",
        "Dominates_Count",
        "Classification",
    ]
]

<IPython.core.display.Javascript object>

In [128]:
df_sorted_topo_models_cols.groupby("Model_bkp_2").describe().round(2)

Dominated_Count                                              \
                      count   mean    std   min   25%   50%   75%   max   
Model_bkp_2                                                               
Bi-LSTM                15.0  47.33  17.75  14.0  39.0  48.0  57.5  72.0   
Conv1D                 15.0  21.53   7.42  11.0  17.0  22.0  26.0  35.0   
LSTM                   15.0  42.60  17.39  17.0  28.5  42.0  55.0  71.0   
MLP                    13.0   7.15   7.69   0.0   3.0   6.0   9.0  30.0   
Transformer            15.0  52.67  11.36  29.0  44.5  55.0  61.5  68.0   

            Dominates_Count                                              
                      count   mean    std   min   25%   50%   75%   max  
Model_bkp_2                                                              
Bi-LSTM                15.0  22.60  17.37   0.0  12.5  21.0  31.5  56.0  
Conv1D                 15.0  47.47   7.77  34.0  43.0  48.0  52.0  59.0  
LSTM                   15.0  27.13  16.01   1.0  15.5  28.0  40.5  50.0  
MLP                    13.0  63.92   8.63  38.0  63.0  66.0  68.0  72.0  
Transformer            15.0  17.73  11.01   3.0   8.5  16.0  26.0  39.0

<IPython.core.display.Javascript object>

In [129]:
df_sorted_topo_models_cols.groupby("Model_bkp_2")["Classification"].describe()

,count,unique,top,freq
Model_bkp_2,,,,
Bi-LSTM,15,2,intermediate_model,14
Conv1D,15,1,intermediate_model,15
LSTM,15,1,intermediate_model,15
MLP,13,2,intermediate_model,12
Transformer,15,1,intermediate_model,15


<IPython.core.display.Javascript object>

In [130]:
df_sorted_topo_models_cols.groupby("Model_bkp_2")[["Classification"]].value_counts()

Model_bkp_2  Classification    
Bi-LSTM      intermediate_model    14
             non_dominant_model     1
Conv1D       intermediate_model    15
LSTM         intermediate_model    15
MLP          intermediate_model    12
             dominant_model         1
Transformer  intermediate_model    15
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [131]:
(
    df_sorted_topo_models_cols.groupby("Model_bkp_2")[["Classification"]]
    .value_counts(normalize=True)
    .to_frame()
    * 100
).round(2)

proportion
Model_bkp_2 Classification                
Bi-LSTM     intermediate_model       93.33
            non_dominant_model        6.67
Conv1D      intermediate_model      100.00
LSTM        intermediate_model      100.00
MLP         intermediate_model       92.31
            dominant_model            7.69
Transformer intermediate_model      100.00

<IPython.core.display.Javascript object>

In [132]:
df_sorted_topo_models_cols

,Model_bkp_2,Dominated_Count,Dominates_Count,Classification
25,MLP,0,72,dominant_model
20,MLP,1,71,intermediate_model
24,MLP,2,70,intermediate_model
19,MLP,3,67,intermediate_model
21,MLP,3,68,intermediate_model
...,...,...,...,...
48,Bi-LSTM,67,3,intermediate_model
66,Transformer,68,3,intermediate_model
51,Bi-LSTM,70,2,intermediate_model
5,LSTM,71,1,intermediate_model


<IPython.core.display.Javascript object>

In [133]:
summary_stats = df_sorted_topo_models_cols.groupby("Model_bkp_2").agg(
    Total_Models=("Model_bkp_2", "count"),
    Mean_Dominance_Count=("Dominates_Count", "mean"),
    Median_Dominance_Count=("Dominates_Count", "median"),
    Total_Dominated=("Dominated_Count", "sum"),
    Total_Dominating=("Dominates_Count", "sum"),
)

total_models = summary_stats[
    "Total_Models"
].sum()  # Get the total number of models across all categories
summary_stats["Dominance Proportion"] = summary_stats["Total_Dominating"] / total_models

<IPython.core.display.Javascript object>

In [134]:
summary_stats

,Total_Models,Mean_Dominance_Count,Median_Dominance_Count,Total_Dominated,Total_Dominating,Dominance Proportion
Model_bkp_2,,,,,,
Bi-LSTM,15,22.600000,21.0,710,339,4.643836
Conv1D,15,47.466667,48.0,323,712,9.753425
LSTM,15,27.133333,28.0,639,407,5.575342
MLP,13,63.923077,66.0,93,831,11.383562
Transformer,15,17.733333,16.0,790,266,3.643836


<IPython.core.display.Javascript object>

In [135]:
df_sorted_topo_models_grouped.T

Timesteps                                                 \
                count      mean       std  min  25%  50%   75%   max   
Model_bkp_2                                                            
Bi-LSTM          15.0  7.333333  5.498918  1.0  1.0  7.0  14.0  14.0   
Conv1D           15.0  7.333333  5.498918  1.0  1.0  7.0  14.0  14.0   
LSTM             15.0  7.333333  5.498918  1.0  1.0  7.0  14.0  14.0   
MLP               0.0       NaN       NaN  NaN  NaN  NaN   NaN   NaN   
Transformer      15.0  7.333333  5.498918  1.0  1.0  7.0  14.0  14.0   

            RMSE_mean            ... Dominated_Count       Dominates_Count  \
                count      mean  ...             75%   max           count   
Model_bkp_2                      ...                                         
Bi-LSTM          15.0  5.296448  ...            57.5  72.0            15.0   
Conv1D           15.0  1.923078  ...            26.0  35.0            15.0   
LSTM             15.0  4.062341  ...            55.0  71.0            15.0   
MLP              13.0  1.657920  ...             9.0  30.0            13.0   
Transformer      15.0  3.185219  ...            61.5  68.0            15.0   

                                                                 
                  mean        std   min   25%   50%   75%   max  
Model_bkp_2                                                      
Bi-LSTM      22.600000  17.373214   0.0  12.5  21.0  31.5  56.0  
Conv1D       47.466667   7.772356  34.0  43.0  48.0  52.0  59.0  
LSTM         27.133333  16.012793   1.0  15.5  28.0  40.5  50.0  
MLP          63.923077   8.626138  38.0  63.0  66.0  68.0  72.0  
Transformer  17.733333  11.009520   3.0   8.5  16.0  26.0  39.0  

[5 rows x 64 columns]

<IPython.core.display.Javascript object>

### SCPM Analysis

In [136]:
cols = [
    "Model",
    # "Model_bkp",
    "Model_bkp_2",
    "Timesteps",
    "RMSE_mean",
    "MAE_mean",
    "MAPE_mean",
    "R2_mean",
    "SCPM",
    "Dominated_Count",
    "Dominates_Count",
    "Classification",
    "Plant",
]

<IPython.core.display.Javascript object>

In [137]:
df_sorted_scpm = df_sorted_topo.sort_values(by="SCPM").copy()

<IPython.core.display.Javascript object>

In [138]:
# (df_sorted_topo[cols].reset_index() == df_sorted_scpm[cols].reset_index()).all(axis=1)
df_sorted_scpm["topo_scmp_order_eq"] = df_sorted_topo.index == df_sorted_scpm.index

<IPython.core.display.Javascript object>

In [139]:
# df_sorted_topo[cols].round(2).head(5)

<IPython.core.display.Javascript object>

In [140]:
df_sorted_scpm[cols + ["topo_scmp_order_eq"]].round(2).head(7)

,Model,Model_bkp_2,Timesteps,RMSE_mean,MAE_mean,MAPE_mean,R2_mean,SCPM,Dominated_Count,Dominates_Count,Classification,Plant,topo_scmp_order_eq
25,MLP_11,MLP,NaN,1.54,1.15,2.69,0.93,-1.45,0,72,dominant_model,B,True
20,MLP_6,MLP,NaN,1.57,1.17,2.70,0.93,-1.44,1,71,intermediate_model,B,True
24,MLP_10,MLP,NaN,1.58,1.17,2.72,0.93,-1.43,2,70,intermediate_model,B,True
21,MLP_7,MLP,NaN,1.59,1.20,2.80,0.93,-1.41,3,68,intermediate_model,B,False
19,MLP_5,MLP,NaN,1.60,1.20,2.80,0.93,-1.41,3,67,intermediate_model,B,False
26,MLP_12,MLP,NaN,1.60,1.21,2.81,0.93,-1.41,4,67,intermediate_model,B,True
15,MLP_1,MLP,NaN,1.63,1.23,2.89,0.93,-1.38,6,66,intermediate_model,B,True


<IPython.core.display.Javascript object>

# Global Analysis (fine tuning results)

# Definitions

# Topological Analysis - NEW

## Definitions

In [141]:
# Function to compare two models strictly based on means only
def dominates(model1, model2):
    """Return True if model1 strictly dominates model2, False otherwise."""
    # Check if model1 is at least as good as model2 in all metrics
    at_least_as_good = (
        model1["RMSE_mean"] <= model2["RMSE_mean"]
        and model1["MAE_mean"] <= model2["MAE_mean"]
        and model1["MAPE_mean"] <= model2["MAPE_mean"]
        and model1["R2_mean"] >= model2["R2_mean"]
    )

    # Check if model1 is strictly better in at least one metric
    strictly_better_in_one = (
        model1["RMSE_mean"] < model2["RMSE_mean"]
        or model1["MAE_mean"] < model2["MAE_mean"]
        or model1["MAPE_mean"] < model2["MAPE_mean"]
        or model1["R2_mean"] > model2["R2_mean"]
    )

    # Model1 strictly dominates Model2 if it is at least as good in all metrics and
    # strictly better in at least one metric
    return at_least_as_good and strictly_better_in_one

<IPython.core.display.Javascript object>

In [142]:
def get_dominance_matrix_and_graph(df):
    n = len(df)
    dominance_matrix = np.zeros((n, n), dtype=bool)

    for i in range(n):
        for j in range(n):
            if i != j:
                dominance_matrix[i, j] = dominates(df.iloc[i], df.iloc[j])

    # Create the dominance graph
    dominance_graph = {i: [] for i in range(n)}
    for i in range(n):
        for j in range(n):
            if dominance_matrix[i, j]:
                dominance_graph[i].append(j)

    return dominance_matrix, dominance_graph

<IPython.core.display.Javascript object>

In [143]:
# Topological Sorting
def topological_sort(graph):
    """Perform topological sorting on the given graph."""
    in_degree = {u: 0 for u in graph}
    for u in graph:
        for v in graph[u]:
            in_degree[v] += 1

    queue = deque([u for u in graph if in_degree[u] == 0])
    topo_order = []

    while queue:
        u = queue.popleft()
        topo_order.append(u)

        for v in graph[u]:
            in_degree[v] -= 1
            if in_degree[v] == 0:
                queue.append(v)

    return topo_order

<IPython.core.display.Javascript object>

In [144]:
# Function to find the top models in each group
def find_top_models(group):
    max_net_dominance = group["Net_Dominance"].max()
    top_models = group[group["Net_Dominance"] == max_net_dominance]
    return top_models

<IPython.core.display.Javascript object>

### Dominance Analysis and SCPM

In [145]:
def compute_scpm(df):
    scaler = StandardScaler()
    standardized_metrics = scaler.fit_transform(
        df[["RMSE_mean", "MAE_mean", "MAPE_mean", "R2_mean"]]
    )

    # Creating a new dataframe with standardized values
    standardized_df = pd.DataFrame(
        standardized_metrics,
        columns=["RMSE_mean", "MAE_mean", "MAPE_mean", "R2_mean"],
    )

    # Summing all metrics and subtracting R2_mean
    standardized_df["Result"] = (
        standardized_df[["RMSE_mean", "MAE_mean", "MAPE_mean"]].sum(axis=1)
        - standardized_df["R2_mean"]
    )

    df["SCPM"] = standardized_df["Result"]
    return df

<IPython.core.display.Javascript object>

In [146]:
def make_dominance_analysis(df):
    # Compute dominance matrix and graph
    dominance_matrix, dominance_graph = get_dominance_matrix_and_graph(df)

    # Get the topological order
    topo_order = topological_sort(dominance_graph)
    df = df.copy()
    n = len(topo_order)

    # Identify Dominant Models
    dominant_models = [
        df["Model_bkp_2"].iloc[i]
        for i in range(len(df))
        if not np.any(dominance_matrix[:, i])
    ]

    # Identify Non-Dominant Models
    non_dominant_models = [
        df["Model_bkp_2"].iloc[i]
        for i in range(len(df))
        if np.any(dominance_matrix[:, i])
    ]

    # Identify Models that are both dominated and dominate at least one
    intermediate_models = [
        df["Model_bkp_2"].iloc[i]
        for i in range(len(df))
        if np.any(dominance_matrix[:, i]) and np.any(dominance_matrix[i, :])
    ]

    # Add count of times each model is dominated by others
    df["Dominated_Count"] = np.sum(dominance_matrix, axis=0)

    # Classify models
    df["Classification"] = "intermediate_model"  # default to intermediate_model
    df.loc[
        ~np.any(dominance_matrix, axis=1) & np.any(dominance_matrix, axis=0),
        "Classification",
    ] = "non_dominant_model"
    df.loc[~np.any(dominance_matrix, axis=0), "Classification"] = "dominant_model"

    # Map the topological order to the model names and get dominance counts
    sorted_models = [df["Model"].iloc[i] for i in topo_order]

    df_sorted = df.iloc[topo_order].copy()
    df_sorted["Dominates_Count"] = dominance_matrix.sum(axis=1)[topo_order]
    df_sorted_topo = df_sorted.copy()

    df_sorted = df_sorted.sort_values(by="Dominates_Count", ascending=False)
    df_sorted_count = df_sorted.copy()

    return {
        "dominance_matrix": dominance_matrix,
        "dominance_graph": dominance_graph,
        "df_sorted_topo": df_sorted_topo,
        "df_sorted_count": df_sorted_count,
    }

<IPython.core.display.Javascript object>

# Reading The files

In [147]:
csv_files_path_fine_tuning_full = dict()
csv_files_path_fine_tuning_grouped = dict()

<IPython.core.display.Javascript object>

## 206

### Plant B

In [148]:
path_fine_tuning_full = "/home/peressim/projects/ccs28-ml-modelling/reports/results/global_models/206/b/fine_tuning/full/"
path_fine_tuning_grouped = "/home/peressim/projects/ccs28-ml-modelling/reports/results/global_models/206/b/fine_tuning/grouped/"
plant = "b"

csv_files_path_fine_tuning_full = read_csv_files_path(
    csv_files_path_fine_tuning_full, path_fine_tuning_full, plant
)

csv_files_path_fine_tuning_grouped = read_csv_files_path(
    csv_files_path_fine_tuning_grouped, path_fine_tuning_grouped, plant
)

<IPython.core.display.Javascript object>

### Reading all data - pre training

#### Full

In [149]:
dfs_plant = dict()
column = ("plant", "")
pos = 2

for plant, csv_files in csv_files_path_fine_tuning_full.items():
    df = read_csv_files_full(csv_files, plant)
    df["plant"] = plant
    dfs_plant[plant] = df

<IPython.core.display.Javascript object>

In [150]:
df_fine_tuning_full = pd.concat(dfs_plant.values(), ignore_index=True)

<IPython.core.display.Javascript object>

In [151]:
df_fine_tuning_full.shape

(720, 23)

<IPython.core.display.Javascript object>

#### Grouped

In [152]:
dfs_plant = dict()
column = ("plant", "")
pos = 2

for plant, csv_files in csv_files_path_fine_tuning_grouped.items():
    df = read_csv_files_grouped(csv_files, plant)
    df["plant"] = plant
    df = preprocess_rename_columns(df)
    df = preprocess_change_columns_order(df, column, pos)
    dfs_plant[plant] = df

<IPython.core.display.Javascript object>

In [153]:
df_fine_tuning_grouped = pd.concat(dfs_plant.values(), ignore_index=True)

<IPython.core.display.Javascript object>

In [154]:
df_fine_tuning_grouped.shape

(60, 25)

<IPython.core.display.Javascript object>

In [155]:
df_copy = df_fine_tuning_grouped.copy()
df_copy = (
    df_copy.reset_index(level=0)
    .rename({"level_0": "Plant"}, axis=1)
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

## Preprocessing steps

In [156]:
patterns = [
    "Chemical + Mineralogical + Feature Engineering",
    "Chemical + Mineralogical + CS7 + One-Hot",
    "Chemical + Mineralogical + Physical + One-Hot",
    "Chemical + Mineralogical + CS3 + One-Hot",
    "Chemical + Mineralogical + CS3 + CS7 + One-Hot",
    "Chemical + Mineralogical + One-Hot",
    "Chemical + Mineralogical + CS1 + CS3 + CS7 + One-Hot",
    "Chemical + Mineralogical + CS1 + CS3 + One-Hot",
    "Chemical + Mineralogical + CS1 + One-Hot",
    "Chemical + Mineralogical + CS2 + One-Hot",
    "Chemical + Feature Engineering",
]

replace_dict = {
    "Chemical + Mineralogical + CS3 + CS7": "Chemical + Mineralogical + Early CS",
    "Chemical + Mineralogical + CS7": "Chemical + Mineralogical + Early CS",
    "Chemical + Mineralogical + CS3": "Chemical + Mineralogical + Early CS",
    "Chemical + Mineralogical + CS1": "Chemical + Mineralogical + Early CS",
    "Chemical + Mineralogical + CS1 + CS3": "Chemical + Mineralogical + Early CS",
    "Chemical + Mineralogical + CS1 + CS3 + CS7": "Chemical + Mineralogical + Early CS",
    "Chemical + Mineralogical + CS2": "Chemical + Mineralogical + Early CS",
    "Chemical + CS1 + CS3 + CS7": "Chemical + Mineralogical + Early CS",
    "Chemical + CS3 + CS7": "Chemical + Mineralogical + Early CS",
    "Chemical + CS7": "Chemical + Mineralogical + Early CS",
    "Chemical + CS3": "Chemical + Mineralogical + Early CS",
    "Chemical + CS1 + CS3": "Chemical + Mineralogical + Early CS",
    "Chemical + CS1": "Chemical + Mineralogical + Early CS",
    "Chemical + CS1 + CS7": "Chemical + Mineralogical + Early CS",
}

replace_dict_2 = {
    "Chemical": "Chemical + Mineralogical",
    "Chemical + Properties CS Less": "Chemical + Mineralogical + Physical - Early CS",
    "Chemical + Mineralogical + Properties CS Less": "Chemical + Mineralogical + Physical - Early CS",
    "Chemical + Physical": "Chemical + Mineralogical + Physical",
}

df_copy = df_copy[~df_copy["Cross Validation"].eq("Out of time Split")].reset_index(
    drop=True
)

# Removing afterwards
df_copy = df_copy[~df_copy["Cross Validation"].eq("Repeated KFold")].reset_index(
    drop=True
)

df_copy = df_copy[~df_copy["Features"].apply(lambda x: x in patterns)].reset_index(
    drop=True
)
df_copy["Features_bkp"] = df_copy["Features"].copy()
df_copy["Features"] = df_copy["Features"].replace(replace_dict)

df_copy["Features"] = df_copy["Features"].replace(replace_dict_2)

# Removing afterwards
# df_copy = df_copy[
#     ~df_copy["Features"].eq("Chemical + Mineralogical + Early CS")
# ].reset_index(drop=True)

<IPython.core.display.Javascript object>

In [157]:
df_copy[("MAPE Train", "mean")] = df_copy[("MAPE Train", "mean")] * 100
df_copy[("MAPE Train", "std")] = df_copy[("MAPE Train", "std")] * 100
df_copy[("MAPE Test", "mean")] = df_copy[("MAPE Test", "mean")] * 100
df_copy[("MAPE Test", "std")] = df_copy[("MAPE Test", "std")] * 100

df_copy["Model_bkp"] = df_copy["Model"].copy()
df_copy["Model_bkp_2"] = df_copy["Model"] + df_copy["Timesteps"].apply(
    lambda x: "" if pd.isna(x) else "_" + str(x)
)
df_copy["Model"] = df_copy["Model"].replace(
    {
        "MLP": "Neural Networks",
        "LSTM": "Neural Networks",
        "GRU": "Neural Networks",
        "BidirectionalLSTM": "Neural Networks",
        "BidirectionalGRU": "Neural Networks",
        "Transformer": "Neural Networks",
        "Decision Tree": "Trees",
        "Random Forest": "Trees",
        "XGBoost": "Trees",
    }
)

<IPython.core.display.Javascript object>

In [158]:
df_copy["Cross Validation"].unique()

array(['Blocking Time Series Split', 'Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [159]:
df_copy["Features"].unique()

array(['Chemical + Mineralogical', 'Chemical + Mineralogical + Physical',
       'Chemical + Mineralogical + Physical - Early CS'], dtype=object)

<IPython.core.display.Javascript object>

In [160]:
df_copy["Model"].unique()

array(['Neural Networks', 'Conv1D'], dtype=object)

<IPython.core.display.Javascript object>

In [161]:
df_copy["Model_bkp"].unique()

array(['Transformer', 'MLP', 'BidirectionalLSTM', 'Conv1D', 'LSTM'],
      dtype=object)

<IPython.core.display.Javascript object>

In [162]:
df_copy["Model_bkp_2"].unique()

array(['Transformer_7.0', 'MLP', 'BidirectionalLSTM_14.0', 'Conv1D_14.0',
       'BidirectionalLSTM_1.0', 'LSTM_1.0', 'LSTM_14.0', 'Conv1D_1.0',
       'Transformer_1.0', 'Conv1D_7.0', 'LSTM_7.0'], dtype=object)

<IPython.core.display.Javascript object>

In [163]:
df_copy.shape

(30, 29)

<IPython.core.display.Javascript object>

In [164]:
df_copy_grouped = df_copy.copy()

<IPython.core.display.Javascript object>

In [165]:
df_copy[
    [
        ("RMSE Test", "mean"),
        ("MAE Test", "mean"),
        ("MAPE Test", "mean"),
        ("R2 Test", "mean"),
    ]
].describe().round(2).T

,,count,mean,std,min,25%,50%,75%,max
RMSE Test,mean,30.0,2.25,0.58,1.53,1.91,2.03,2.39,3.96
MAE Test,mean,30.0,1.74,0.50,1.17,1.42,1.57,1.87,3.17
MAPE Test,mean,30.0,3.95,1.11,2.62,3.24,3.62,4.27,7.09
R2 Test,mean,30.0,-0.27,1.00,-3.31,-0.41,0.01,0.40,0.65


<IPython.core.display.Javascript object>

In [166]:
df_copy["plant"].unique()

array(['b'], dtype=object)

<IPython.core.display.Javascript object>

# Global Analysis (fine tuning results)

## ECICS - 206

In [167]:
df_ecics_grouped = df_copy.copy()

<IPython.core.display.Javascript object>

In [168]:
df_results = df_ecics_grouped[
    [
        ("Company", ""),
        ("plant", ""),
        ("Cross Validation", ""),
        ("Features", ""),
        ('Features_bkp', ''),
        ("Model", ""),
        ("Model_bkp", ""),         
        ('Model_bkp_2',''),
        ("Timesteps", ""),
        ("RMSE Test", "mean"),
        ("MAE Test", "mean"),
        ("MAPE Test", "mean"),
        ("R2 Test", "mean"),
        ("RMSE Test", "std"),
        ("MAE Test", "std"),
        ("MAPE Test", "std"),
        ("R2 Test", "std"),
    ]
].copy()
df_results = df_results.reset_index(drop=True)

# Rename columns
new_column_names = [
    "Company",
    "Plant",
    "Cross Validation",
    "Features",
    "Features_bkp",
    "Model",
    "Model_bkp",
    "Model_bkp_2",
    "Timesteps",
    "RMSE_mean",
    "MAE_mean",
    "MAPE_mean",
    "R2_mean",
    "RMSE_std",
    "MAE_std",
    "MAPE_std",
    "R2_std",
]

df_results.columns = new_column_names


<IPython.core.display.Javascript object>

In [169]:
df_results.shape

(30, 17)

<IPython.core.display.Javascript object>

In [170]:
df_results["Cross Validation"].unique()

array(['Blocking Time Series Split', 'Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

## Variable Grouping: CM

#### Analysis of the best models identified in the Local Analysis

**Best models for variable groupping CM were from:**

*BTiSS:*
    Dominance analysis: Plant B
    SCPM:Plant B

*TiSS:*
    Dominance analysis: Plant B
    SCPM:Plant B

In [171]:
df_results["Features"].unique()

array(['Chemical + Mineralogical', 'Chemical + Mineralogical + Physical',
       'Chemical + Mineralogical + Physical - Early CS'], dtype=object)

<IPython.core.display.Javascript object>

### Plant B

In [172]:
df_results_cm = (
    df_results[df_results["Features"].eq("Chemical + Mineralogical")]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [173]:
df_results_cm["Features"].unique()

array(['Chemical + Mineralogical'], dtype=object)

<IPython.core.display.Javascript object>

In [174]:
df_results_cm.shape

(10, 17)

<IPython.core.display.Javascript object>

In [175]:
df_results_cm = df_results_cm[df_results_cm["Plant"].eq("b")].reset_index(drop=True)

<IPython.core.display.Javascript object>

In [176]:
df_results_cm.shape

(10, 17)

<IPython.core.display.Javascript object>

### Blocking time series

In [177]:
df_results_cm["Cross Validation"].unique()

array(['Blocking Time Series Split', 'Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [178]:
df_results_cm_btss = (
    df_results_cm[df_results_cm["Cross Validation"].eq("Blocking Time Series Split")]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [179]:
df_results_cm_btss["Cross Validation"].unique()

array(['Blocking Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [180]:
df_results_cm_btss.shape

(5, 17)

<IPython.core.display.Javascript object>

###### SCPM computation

In [181]:
df_results_cm_btss = compute_scpm(df_results_cm_btss)

<IPython.core.display.Javascript object>

##### Dominance Analysis

In [182]:
# Per project
dominance_dict = make_dominance_analysis(df_results_cm_btss)
dominance_matrix_cm_btss = dominance_dict["dominance_matrix"]
dominance_graph_cm_btss = dominance_dict["dominance_graph"]
df_sorted_count = dominance_dict["df_sorted_count"]
df_sorted_topo = dominance_dict["df_sorted_topo"]

<IPython.core.display.Javascript object>

In [183]:
dominance_matrix_cm_btss.shape, len(dominance_graph_cm_btss)

((5, 5), 5)

<IPython.core.display.Javascript object>

In [184]:
df_sorted_topo["Classification"].value_counts()

Classification
dominant_model        2
intermediate_model    2
non_dominant_model    1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [185]:
df_sorted_topo.to_csv(
    "../../../../reports/results/global_models/ecics/ecics_206_b_dominance_analysis_cm_btss.csv",
    index=False,
)

<IPython.core.display.Javascript object>

In [186]:
df_sorted_topo.head(5)

,Company,Plant,Cross Validation,Features,Features_bkp,Model,Model_bkp,Model_bkp_2,Timesteps,RMSE_mean,...,MAPE_mean,R2_mean,RMSE_std,MAE_std,MAPE_std,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count
3,206,b,Blocking Time Series Split,Chemical + Mineralogical,Chemical,Neural Networks,MLP,MLP,NaN,2.462526,...,4.151863,-1.056172,0.700195,0.459580,1.298147,2.552216,-2.841227,0,dominant_model,2
4,206,b,Blocking Time Series Split,Chemical + Mineralogical,Chemical,Neural Networks,LSTM,LSTM_14.0,14.0,2.416251,...,4.543914,-0.088879,0.653714,0.530939,1.351213,0.556855,-3.187693,0,dominant_model,3
0,206,b,Blocking Time Series Split,Chemical + Mineralogical,Chemical,Neural Networks,Transformer,Transformer_7.0,7.0,2.429835,...,4.700572,-0.432805,1.189071,1.056618,2.887948,1.324846,-2.644004,1,intermediate_model,2
2,206,b,Blocking Time Series Split,Chemical + Mineralogical,Chemical,Conv1D,Conv1D,Conv1D_14.0,14.0,3.075219,...,5.756775,-1.473464,0.605005,0.597842,1.441186,2.133071,1.667540,3,intermediate_model,1
1,206,b,Blocking Time Series Split,Chemical + Mineralogical,Chemical,Neural Networks,BidirectionalLSTM,BidirectionalLSTM_14.0,14.0,3.756990,...,7.092079,-2.902393,1.390475,1.240027,2.777846,4.203490,7.005384,4,non_dominant_model,0


<IPython.core.display.Javascript object>

In [187]:
df_sorted_topo_cols = df_sorted_topo[
    [
        # "Model",
        # "Model_bkp",
        "Model_bkp_2",
        "RMSE_mean",
        "RMSE_std",
        "MAE_mean",
        "MAE_std",
        "MAPE_mean",
        "MAPE_std",
        "R2_mean",
        "R2_std",
        "SCPM",
        "Dominated_Count",
        "Classification",
        "Dominates_Count",
        "Plant",
    ]
].round(2)

<IPython.core.display.Javascript object>

###### Dominant Models

In [188]:
# Top 4
df_sorted_topo_cols.sort_values(
    by=["Dominates_Count", "SCPM"], ascending=[False, True]
).iloc[0:2]

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
4,LSTM_14.0,2.42,0.65,1.98,0.53,4.54,1.35,-0.09,0.56,-3.19,0,dominant_model,3,b
3,MLP,2.46,0.70,1.81,0.46,4.15,1.30,-1.06,2.55,-2.84,0,dominant_model,2,b


<IPython.core.display.Javascript object>

In [189]:
df_sorted_topo_cols[df_sorted_topo_cols["Classification"].eq("dominant_model")].shape

(2, 14)

<IPython.core.display.Javascript object>

###### Intermediate models

In [190]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("intermediate_model")
].sort_values(by=["Dominates_Count", "SCPM"], ascending=[False, True]).iloc[:3]

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
0,Transformer_7.0,2.43,1.19,1.99,1.06,4.70,2.89,-0.43,1.32,-2.64,1,intermediate_model,2,b
2,Conv1D_14.0,3.08,0.61,2.51,0.60,5.76,1.44,-1.47,2.13,1.67,3,intermediate_model,1,b


<IPython.core.display.Javascript object>

In [191]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("intermediate_model")
].shape

(2, 14)

<IPython.core.display.Javascript object>

###### non-dominant models

In [192]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("non_dominant_model")
].sort_values(by=["Dominates_Count", "SCPM"], ascending=[False, True]).iloc[:2]

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
1,BidirectionalLSTM_14.0,3.76,1.39,3.17,1.24,7.09,2.78,-2.9,4.2,7.01,4,non_dominant_model,0,b


<IPython.core.display.Javascript object>

In [193]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("non_dominant_model")
].shape

(1, 14)

<IPython.core.display.Javascript object>

###### Statistics per model type

In [194]:
df_sorted_topo_models_cols = df_sorted_topo[
    [
        "Model",
        # "Model_bkp",
        # "Model_bkp_2",
        "RMSE_mean",
        "RMSE_std",
        "MAE_mean",
        "MAE_std",
        "MAPE_mean",
        "MAPE_std",
        "R2_mean",
        "R2_std",
        "SCPM",
        "Dominated_Count",
        "Dominates_Count",
        "Classification",
        "Plant",
    ]
]  # .round(2)

<IPython.core.display.Javascript object>

In [195]:
df_sorted_topo["Model"].value_counts()

Model
Neural Networks    4
Conv1D             1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [196]:
df_sorted_topo_models_grouped = df_sorted_topo_models_cols.groupby("Model").describe().T

<IPython.core.display.Javascript object>

In [197]:
# pd.set_option("display.max_rows", None)
# df_sorted_topo_models_grouped

<IPython.core.display.Javascript object>

In [198]:
df_sorted_topo_models_cols = df_sorted_topo[
    [
        "Model",
        "Dominated_Count",
        "Dominates_Count",
        "Classification",
    ]
]

<IPython.core.display.Javascript object>

In [199]:
(
    df_sorted_topo_models_cols.groupby("Model")[["Classification"]]
    .value_counts(normalize=True)
    .to_frame()
    * 100
).round(2)

proportion
Model           Classification                
Conv1D          intermediate_model       100.0
Neural Networks dominant_model            50.0
                intermediate_model        25.0
                non_dominant_model        25.0

<IPython.core.display.Javascript object>

In [200]:
df_sorted_topo_models_cols.groupby("Model").describe().round(2).T

Model                  Conv1D  Neural Networks
Dominated_Count count     1.0             4.00
                mean      3.0             1.25
                std       NaN             1.89
                min       3.0             0.00
                25%       3.0             0.00
                50%       3.0             0.50
                75%       3.0             1.75
                max       3.0             4.00
Dominates_Count count     1.0             4.00
                mean      1.0             1.75
                std       NaN             1.26
                min       1.0             0.00
                25%       1.0             1.50
                50%       1.0             2.00
                75%       1.0             2.25
                max       1.0             3.00

<IPython.core.display.Javascript object>

In [201]:
# df_sorted_topo_models_cols

<IPython.core.display.Javascript object>

In [202]:
df_sorted_topo_models_cols.groupby("Model")["Classification"].describe()

,count,unique,top,freq
Model,,,,
Conv1D,1,1,intermediate_model,1
Neural Networks,4,3,dominant_model,2


<IPython.core.display.Javascript object>

In [203]:
df_sorted_topo_models_cols.groupby("Model")[["Classification"]].value_counts()

Model            Classification    
Conv1D           intermediate_model    1
Neural Networks  dominant_model        2
                 intermediate_model    1
                 non_dominant_model    1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [204]:
# df_sorted_topo_models_cols

<IPython.core.display.Javascript object>

In [205]:
summary_stats = df_sorted_topo_models_cols.groupby("Model").agg(
    Total_Models=("Model", "count"),
    Mean_Dominance_Count=("Dominates_Count", "mean"),
    Median_Dominance_Count=("Dominates_Count", "median"),
    Total_Dominated=("Dominated_Count", "sum"),
    Total_Dominating=("Dominates_Count", "sum"),
)

total_models = summary_stats[
    "Total_Models"
].sum()  # Get the total number of models across all categories
summary_stats["Dominance Proportion"] = summary_stats["Total_Dominating"] / total_models

<IPython.core.display.Javascript object>

In [206]:
summary_stats

,Total_Models,Mean_Dominance_Count,Median_Dominance_Count,Total_Dominated,Total_Dominating,Dominance Proportion
Model,,,,,,
Conv1D,1,1.00,1.0,3,1,0.2
Neural Networks,4,1.75,2.0,5,7,1.4


<IPython.core.display.Javascript object>

In [207]:
df_sorted_topo_models_grouped.T

RMSE_mean                                                    \
                    count      mean       std       min       25%       50%   
Model                                                                         
Conv1D                1.0  3.075219       NaN  3.075219  3.075219  3.075219   
Neural Networks       4.0  2.766400  0.660678  2.416251  2.426439  2.446180   

                                    RMSE_std            ... Dominated_Count  \
                      75%       max    count      mean  ...             75%   
Model                                                   ...                   
Conv1D           3.075219  3.075219      1.0  0.605005  ...            3.00   
Neural Networks  2.786142  3.756990      4.0  0.983364  ...            1.75   

                     Dominates_Count                                            
                 max           count  mean       std  min  25%  50%   75%  max  
Model                                                                           
Conv1D           3.0             1.0  1.00       NaN  1.0  1.0  1.0  1.00  1.0  
Neural Networks  4.0             4.0  1.75  1.258306  0.0  1.5  2.0  2.25  3.0  

[2 rows x 88 columns]

<IPython.core.display.Javascript object>

### SCPM Analysis

In [208]:
cols = [
    # "Model",
    # "Model_bkp",
    "Model_bkp_2",
    "RMSE_mean",
    "RMSE_std",
    "MAE_mean",
    "MAE_std",
    "MAPE_mean",
    "MAPE_std",
    "R2_mean",
    "R2_std",
    "SCPM",
    "Dominated_Count",
    "Dominates_Count",
    "Classification",
    "Plant",
]

<IPython.core.display.Javascript object>

In [209]:
df_sorted_scpm = df_sorted_topo.sort_values(by="SCPM").copy()

<IPython.core.display.Javascript object>

In [210]:
# (df_sorted_topo[cols].reset_index() == df_sorted_scpm[cols].reset_index()).all(axis=1)
df_sorted_scpm["topo_scmp_order_eq"] = df_sorted_topo.index == df_sorted_scpm.index

<IPython.core.display.Javascript object>

In [211]:
df_sorted_scpm[cols + ["topo_scmp_order_eq"]].round(2).head(5)

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Dominates_Count,Classification,Plant,topo_scmp_order_eq
4,LSTM_14.0,2.42,0.65,1.98,0.53,4.54,1.35,-0.09,0.56,-3.19,0,3,dominant_model,b,False
3,MLP,2.46,0.70,1.81,0.46,4.15,1.30,-1.06,2.55,-2.84,0,2,dominant_model,b,False
0,Transformer_7.0,2.43,1.19,1.99,1.06,4.70,2.89,-0.43,1.32,-2.64,1,2,intermediate_model,b,True
2,Conv1D_14.0,3.08,0.61,2.51,0.60,5.76,1.44,-1.47,2.13,1.67,3,1,intermediate_model,b,True
1,BidirectionalLSTM_14.0,3.76,1.39,3.17,1.24,7.09,2.78,-2.90,4.20,7.01,4,0,non_dominant_model,b,True


<IPython.core.display.Javascript object>

### Time Series Split

In [212]:
df_results_cm = (
    df_results[df_results["Features"].eq("Chemical + Mineralogical")]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [213]:
df_results_cm["Features"].unique()

array(['Chemical + Mineralogical'], dtype=object)

<IPython.core.display.Javascript object>

In [214]:
df_results_cm.shape

(10, 17)

<IPython.core.display.Javascript object>

In [215]:
df_results_cm.shape

(10, 17)

<IPython.core.display.Javascript object>

In [216]:
df_results_cm["Cross Validation"].unique()

array(['Blocking Time Series Split', 'Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [217]:
df_results_cm_tss = (
    df_results_cm[df_results_cm["Cross Validation"].eq("Time Series Split")]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [218]:
df_results_cm_tss["Cross Validation"].unique()

array(['Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [219]:
df_results_cm_tss.shape

(5, 17)

<IPython.core.display.Javascript object>

###### SCPM computation

In [220]:
df_results_cm_tss = compute_scpm(df_results_cm_tss)

<IPython.core.display.Javascript object>

##### Dominance Analysis

In [221]:
# Per project
dominance_dict = make_dominance_analysis(df_results_cm_tss)
dominance_matrix_cm_tss = dominance_dict["dominance_matrix"]
dominance_graph_cm_tss = dominance_dict["dominance_graph"]
df_sorted_count = dominance_dict["df_sorted_count"]
df_sorted_topo = dominance_dict["df_sorted_topo"]

<IPython.core.display.Javascript object>

In [222]:
dominance_matrix_cm_tss.shape, len(dominance_graph_cm_tss)

((5, 5), 5)

<IPython.core.display.Javascript object>

In [223]:
df_sorted_topo["Classification"].value_counts()

Classification
dominant_model        2
intermediate_model    2
non_dominant_model    1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [224]:
df_sorted_topo.to_csv(
    "../../../../reports/results/global_models/ecics/ecics_206_b_dominance_analysis_cm_tss.csv",
    index=False,
)

<IPython.core.display.Javascript object>

In [225]:
df_sorted_topo.head(5)

,Company,Plant,Cross Validation,Features,Features_bkp,Model,Model_bkp,Model_bkp_2,Timesteps,RMSE_mean,...,MAPE_mean,R2_mean,RMSE_std,MAE_std,MAPE_std,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count
0,206,b,Time Series Split,Chemical + Mineralogical,Chemical,Neural Networks,Transformer,Transformer_7.0,7.0,2.008391,...,3.510177,0.396141,0.588145,0.465469,1.080880,0.218548,-2.643177,0,dominant_model,3
3,206,b,Time Series Split,Chemical + Mineralogical,Chemical,Neural Networks,MLP,MLP,NaN,1.998744,...,3.405980,0.388224,0.459175,0.335112,0.840300,0.255928,-2.804388,0,dominant_model,3
4,206,b,Time Series Split,Chemical + Mineralogical,Chemical,Neural Networks,LSTM,LSTM_14.0,14.0,2.131566,...,3.674771,0.266554,0.366983,0.268187,0.679270,0.293120,-2.065741,2,intermediate_model,2
2,206,b,Time Series Split,Chemical + Mineralogical,Chemical,Conv1D,Conv1D,Conv1D_14.0,14.0,2.451850,...,4.377552,0.043043,0.576346,0.546312,1.233349,0.376729,-0.270186,3,intermediate_model,1
1,206,b,Time Series Split,Chemical + Mineralogical,Chemical,Neural Networks,BidirectionalLSTM,BidirectionalLSTM_14.0,14.0,3.961235,...,6.738696,-1.889462,1.173405,0.914934,1.911753,1.861533,7.783491,4,non_dominant_model,0


<IPython.core.display.Javascript object>

In [226]:
df_sorted_topo_cols = df_sorted_topo[
    [
        # "Model",
        # "Model_bkp",
        "Model_bkp_2",
        "RMSE_mean",
        "RMSE_std",
        "MAE_mean",
        "MAE_std",
        "MAPE_mean",
        "MAPE_std",
        "R2_mean",
        "R2_std",
        "SCPM",
        "Dominated_Count",
        "Classification",
        "Dominates_Count",
        "Plant",
    ]
]  # .round(2)

<IPython.core.display.Javascript object>

###### Dominant Models

In [227]:
# Top 4
df_sorted_topo_cols.iloc[0:2].sort_values(
    by=["Dominates_Count", "SCPM"], ascending=[False, True]
).round(2)

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
3,MLP,2.00,0.46,1.51,0.34,3.41,0.84,0.39,0.26,-2.80,0,dominant_model,3,b
0,Transformer_7.0,2.01,0.59,1.55,0.47,3.51,1.08,0.40,0.22,-2.64,0,dominant_model,3,b


<IPython.core.display.Javascript object>

In [228]:
# Top 4
df_sorted_topo_cols.iloc[0:2].sort_values(
    by=["Dominates_Count", "SCPM"], ascending=[False, True]
).iloc[:4].round(2).round(2)

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
3,MLP,2.00,0.46,1.51,0.34,3.41,0.84,0.39,0.26,-2.80,0,dominant_model,3,b
0,Transformer_7.0,2.01,0.59,1.55,0.47,3.51,1.08,0.40,0.22,-2.64,0,dominant_model,3,b


<IPython.core.display.Javascript object>

In [229]:
df_sorted_topo_cols[df_sorted_topo_cols["Classification"].eq("dominant_model")].shape

(2, 14)

<IPython.core.display.Javascript object>

###### Intermediate models

In [230]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("intermediate_model")
].sort_values(by=["Dominates_Count", "SCPM"], ascending=[False, True]).iloc[:4].round(2)

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
4,LSTM_14.0,2.13,0.37,1.63,0.27,3.67,0.68,0.27,0.29,-2.07,2,intermediate_model,2,b
2,Conv1D_14.0,2.45,0.58,1.95,0.55,4.38,1.23,0.04,0.38,-0.27,3,intermediate_model,1,b


<IPython.core.display.Javascript object>

In [231]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("intermediate_model")
].shape

(2, 14)

<IPython.core.display.Javascript object>

###### non-dominant models

In [232]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("non_dominant_model")
].sort_values(by=["Dominates_Count", "SCPM"], ascending=[False, True]).iloc[:3]

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
1,BidirectionalLSTM_14.0,3.961235,1.173405,3.073194,0.914934,6.738696,1.911753,-1.889462,1.861533,7.783491,4,non_dominant_model,0,b


<IPython.core.display.Javascript object>

In [233]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("non_dominant_model")
].shape

(1, 14)

<IPython.core.display.Javascript object>

###### Statistics per model type

In [234]:
df_sorted_topo_models_cols = df_sorted_topo[
    [
        "Model",
        # "Model_bkp",
        # "Model_bkp_2",
        "RMSE_mean",
        "RMSE_std",
        "MAE_mean",
        "MAE_std",
        "MAPE_mean",
        "MAPE_std",
        "R2_mean",
        "R2_std",
        "SCPM",
        "Dominated_Count",
        "Dominates_Count",
        "Classification",
        "Plant",
    ]
]  # .round(2)

<IPython.core.display.Javascript object>

In [235]:
df_sorted_topo["Model"].value_counts()

Model
Neural Networks    4
Conv1D             1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [236]:
df_sorted_topo_models_grouped = df_sorted_topo_models_cols.groupby("Model").describe().T

<IPython.core.display.Javascript object>

In [237]:
df_sorted_topo_models_cols = df_sorted_topo[
    [
        "Model",
        "Dominated_Count",
        "Dominates_Count",
        "Classification",
    ]
]

<IPython.core.display.Javascript object>

In [238]:
(
    df_sorted_topo_models_cols.groupby("Model")[["Classification"]]
    .value_counts(normalize=True)
    .to_frame()
    * 100
).round(2)

proportion
Model           Classification                
Conv1D          intermediate_model       100.0
Neural Networks dominant_model            50.0
                intermediate_model        25.0
                non_dominant_model        25.0

<IPython.core.display.Javascript object>

In [239]:
df_sorted_topo_models_cols.groupby("Model").describe().round(2).T

Model                  Conv1D  Neural Networks
Dominated_Count count     1.0             4.00
                mean      3.0             1.50
                std       NaN             1.91
                min       3.0             0.00
                25%       3.0             0.00
                50%       3.0             1.00
                75%       3.0             2.50
                max       3.0             4.00
Dominates_Count count     1.0             4.00
                mean      1.0             2.00
                std       NaN             1.41
                min       1.0             0.00
                25%       1.0             1.50
                50%       1.0             2.50
                75%       1.0             3.00
                max       1.0             3.00

<IPython.core.display.Javascript object>

In [240]:
# df_sorted_topo_models_cols

<IPython.core.display.Javascript object>

In [241]:
df_sorted_topo_models_cols.groupby("Model")["Classification"].describe()

,count,unique,top,freq
Model,,,,
Conv1D,1,1,intermediate_model,1
Neural Networks,4,3,dominant_model,2


<IPython.core.display.Javascript object>

In [242]:
df_sorted_topo_models_cols.groupby("Model")[["Classification"]].value_counts()

Model            Classification    
Conv1D           intermediate_model    1
Neural Networks  dominant_model        2
                 intermediate_model    1
                 non_dominant_model    1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [243]:
summary_stats = df_sorted_topo_models_cols.groupby("Model").agg(
    Total_Models=("Model", "count"),
    Mean_Dominance_Count=("Dominates_Count", "mean"),
    Median_Dominance_Count=("Dominates_Count", "median"),
    Total_Dominated=("Dominated_Count", "sum"),
    Total_Dominating=("Dominates_Count", "sum"),
)

total_models = summary_stats[
    "Total_Models"
].sum()  # Get the total number of models across all categories
summary_stats["Dominance Proportion"] = summary_stats["Total_Dominating"] / total_models

<IPython.core.display.Javascript object>

In [244]:
summary_stats.round(2)

,Total_Models,Mean_Dominance_Count,Median_Dominance_Count,Total_Dominated,Total_Dominating,Dominance Proportion
Model,,,,,,
Conv1D,1,1.0,1.0,3,1,0.2
Neural Networks,4,2.0,2.5,6,8,1.6


<IPython.core.display.Javascript object>

In [245]:
df_sorted_topo_models_grouped.T

RMSE_mean                                                    \
                    count      mean       std       min       25%       50%   
Model                                                                         
Conv1D                1.0  2.451850       NaN  2.451850  2.451850  2.451850   
Neural Networks       4.0  2.524984  0.959408  1.998744  2.005979  2.069978   

                                    RMSE_std            ... Dominated_Count  \
                      75%       max    count      mean  ...             75%   
Model                                                   ...                   
Conv1D           2.451850  2.451850      1.0  0.576346  ...             3.0   
Neural Networks  2.588983  3.961235      4.0  0.646927  ...             2.5   

                     Dominates_Count                                          
                 max           count mean       std  min  25%  50%  75%  max  
Model                                                                         
Conv1D           3.0             1.0  1.0       NaN  1.0  1.0  1.0  1.0  1.0  
Neural Networks  4.0             4.0  2.0  1.414214  0.0  1.5  2.5  3.0  3.0  

[2 rows x 88 columns]

<IPython.core.display.Javascript object>

### SCPM Analysis

In [246]:
cols = [
    # "Model",
    # "Model_bkp",
    "Model_bkp_2",
    "RMSE_mean",
    "RMSE_std",
    "MAE_mean",
    "MAE_std",
    "MAPE_mean",
    "MAPE_std",
    "R2_mean",
    "R2_std",
    "SCPM",
    "Dominated_Count",
    "Dominates_Count",
    "Classification",
    "Plant",
]

<IPython.core.display.Javascript object>

#### SCPM Analysis by Project

In [247]:
df_sorted_scpm = df_sorted_topo.sort_values(by="SCPM").copy()

<IPython.core.display.Javascript object>

In [248]:
# (df_sorted_topo[cols].reset_index() == df_sorted_scpm[cols].reset_index()).all(axis=1)
df_sorted_scpm["topo_scmp_order_eq"] = df_sorted_topo.index == df_sorted_scpm.index

<IPython.core.display.Javascript object>

In [249]:
df_sorted_scpm[cols + ["topo_scmp_order_eq"]].round(2).head(5)

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Dominates_Count,Classification,Plant,topo_scmp_order_eq
3,MLP,2.00,0.46,1.51,0.34,3.41,0.84,0.39,0.26,-2.80,0,3,dominant_model,b,False
0,Transformer_7.0,2.01,0.59,1.55,0.47,3.51,1.08,0.40,0.22,-2.64,0,3,dominant_model,b,False
4,LSTM_14.0,2.13,0.37,1.63,0.27,3.67,0.68,0.27,0.29,-2.07,2,2,intermediate_model,b,True
2,Conv1D_14.0,2.45,0.58,1.95,0.55,4.38,1.23,0.04,0.38,-0.27,3,1,intermediate_model,b,True
1,BidirectionalLSTM_14.0,3.96,1.17,3.07,0.91,6.74,1.91,-1.89,1.86,7.78,4,0,non_dominant_model,b,True


<IPython.core.display.Javascript object>

## Variable Grouping: CM-P

#### Analysis of the best models identified in the Local Analysis

**Best models for variable groupping CM-P were from:**

*BTiSS:*
    Dominance analysis: Plant B
    SCPM:Plant B

*TiSS:*
    Dominance analysis: Plant B
    SCPM:Plant B

In [250]:
df_results["Features"].unique()

array(['Chemical + Mineralogical', 'Chemical + Mineralogical + Physical',
       'Chemical + Mineralogical + Physical - Early CS'], dtype=object)

<IPython.core.display.Javascript object>

In [251]:
df_results_cm_p = (
    df_results[
        df_results["Features"].eq("Chemical + Mineralogical + Physical - Early CS")
    ]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [252]:
df_results_cm_p["Features"].unique()

array(['Chemical + Mineralogical + Physical - Early CS'], dtype=object)

<IPython.core.display.Javascript object>

In [253]:
df_results_cm_p.shape

(10, 17)

<IPython.core.display.Javascript object>

### Blocking time series

In [254]:
df_results_cm_p["Cross Validation"].unique()

array(['Blocking Time Series Split', 'Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [255]:
df_results_cm_p_btss = (
    df_results_cm_p[
        df_results_cm_p["Cross Validation"].eq("Blocking Time Series Split")
    ]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [256]:
df_results_cm_p_btss["Cross Validation"].unique()

array(['Blocking Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [257]:
df_results_cm_p_btss.shape

(5, 17)

<IPython.core.display.Javascript object>

###### SCPM computation

In [258]:
df_results_cm_p_btss = compute_scpm(df_results_cm_p_btss)

<IPython.core.display.Javascript object>

##### Dominance Analysis

In [259]:
# Per project
dominance_dict = make_dominance_analysis(df_results_cm_p_btss)
dominance_matrix_cm_p_btss = dominance_dict["dominance_matrix"]
dominance_graph_cm_p_btss = dominance_dict["dominance_graph"]
df_sorted_count = dominance_dict["df_sorted_count"]
df_sorted_topo = dominance_dict["df_sorted_topo"]

<IPython.core.display.Javascript object>

In [260]:
dominance_matrix_cm_p_btss.shape, len(dominance_graph_cm_p_btss)

((5, 5), 5)

<IPython.core.display.Javascript object>

In [261]:
df_sorted_topo["Classification"].value_counts()

Classification
dominant_model        2
intermediate_model    2
non_dominant_model    1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [262]:
df_sorted_topo.to_csv(
    "../../../../reports/results/global_models/ecics/ecics_206_b_dominance_analysis_cm_p_btss.csv",
    index=False,
)

<IPython.core.display.Javascript object>

In [263]:
df_sorted_topo.head(5)

,Company,Plant,Cross Validation,Features,Features_bkp,Model,Model_bkp,Model_bkp_2,Timesteps,RMSE_mean,...,MAPE_mean,R2_mean,RMSE_std,MAE_std,MAPE_std,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count
1,206,b,Blocking Time Series Split,Chemical + Mineralogical + Physical - Early CS,Chemical + Properties CS Less,Neural Networks,BidirectionalLSTM,BidirectionalLSTM_1.0,1.0,2.104867,...,3.840031,-0.049865,0.900549,0.724796,2.034133,0.868114,-2.076545,0,dominant_model,2
4,206,b,Blocking Time Series Split,Chemical + Mineralogical + Physical - Early CS,Chemical + Properties CS Less,Neural Networks,Transformer,Transformer_1.0,1.0,1.987151,...,3.557103,-0.121805,0.542622,0.380991,1.108645,1.125835,-2.808265,0,dominant_model,3
0,206,b,Blocking Time Series Split,Chemical + Mineralogical + Physical - Early CS,Chemical + Properties CS Less,Neural Networks,MLP,MLP,NaN,2.043694,...,3.722352,-0.175799,0.826130,0.633926,1.767099,1.102127,-2.335072,1,intermediate_model,2
3,206,b,Blocking Time Series Split,Chemical + Mineralogical + Physical - Early CS,Chemical + Properties CS Less,Conv1D,Conv1D,Conv1D_7.0,7.0,2.289793,...,4.217535,-0.341420,0.543307,0.436485,1.253815,1.219415,-0.620539,3,intermediate_model,1
2,206,b,Blocking Time Series Split,Chemical + Mineralogical + Physical - Early CS,Chemical + Properties CS Less,Neural Networks,LSTM,LSTM_1.0,1.0,3.406710,...,5.983505,-3.306592,0.941637,0.945964,1.877023,5.938426,7.840421,4,non_dominant_model,0


<IPython.core.display.Javascript object>

In [264]:
df_sorted_topo_cols = df_sorted_topo[
    [
        # "Model",
        # "Model_bkp",
        "Model_bkp_2",
        "RMSE_mean",
        "RMSE_std",
        "MAE_mean",
        "MAE_std",
        "MAPE_mean",
        "MAPE_std",
        "R2_mean",
        "R2_std",
        "SCPM",
        "Dominated_Count",
        "Classification",
        "Dominates_Count",
        "Plant",
    ]
].round(2)

<IPython.core.display.Javascript object>

###### Dominant Models

In [265]:
# Top 4
df_sorted_topo_cols.sort_values(
    by=["Dominates_Count", "SCPM"], ascending=[False, True]
).iloc[0:5]

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
4,Transformer_1.0,1.99,0.54,1.54,0.38,3.56,1.11,-0.12,1.13,-2.81,0,dominant_model,3,b
0,MLP,2.04,0.83,1.60,0.63,3.72,1.77,-0.18,1.10,-2.34,1,intermediate_model,2,b
1,BidirectionalLSTM_1.0,2.10,0.90,1.64,0.72,3.84,2.03,-0.05,0.87,-2.08,0,dominant_model,2,b
3,Conv1D_7.0,2.29,0.54,1.83,0.44,4.22,1.25,-0.34,1.22,-0.62,3,intermediate_model,1,b
2,LSTM_1.0,3.41,0.94,2.68,0.95,5.98,1.88,-3.31,5.94,7.84,4,non_dominant_model,0,b


<IPython.core.display.Javascript object>

In [266]:
df_sorted_topo_cols[df_sorted_topo_cols["Classification"].eq("dominant_model")].shape

(2, 14)

<IPython.core.display.Javascript object>

###### Intermediate models

In [267]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("intermediate_model")
].sort_values(by=["Dominates_Count", "SCPM"], ascending=[False, True]).iloc[:3]

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
0,MLP,2.04,0.83,1.60,0.63,3.72,1.77,-0.18,1.10,-2.34,1,intermediate_model,2,b
3,Conv1D_7.0,2.29,0.54,1.83,0.44,4.22,1.25,-0.34,1.22,-0.62,3,intermediate_model,1,b


<IPython.core.display.Javascript object>

In [268]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("intermediate_model")
].shape

(2, 14)

<IPython.core.display.Javascript object>

###### non-dominant models

In [269]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("non_dominant_model")
].sort_values(by=["Dominates_Count", "SCPM"], ascending=[False, True]).iloc[:2]

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
2,LSTM_1.0,3.41,0.94,2.68,0.95,5.98,1.88,-3.31,5.94,7.84,4,non_dominant_model,0,b


<IPython.core.display.Javascript object>

In [270]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("non_dominant_model")
].shape

(1, 14)

<IPython.core.display.Javascript object>

###### Statistics per model type

In [271]:
df_sorted_topo_models_cols = df_sorted_topo[
    [
        "Model",
        # "Model_bkp",
        # "Model_bkp_2",
        "RMSE_mean",
        "RMSE_std",
        "MAE_mean",
        "MAE_std",
        "MAPE_mean",
        "MAPE_std",
        "R2_mean",
        "R2_std",
        "SCPM",
        "Dominated_Count",
        "Dominates_Count",
        "Classification",
        "Plant",
    ]
]  # .round(2)

<IPython.core.display.Javascript object>

In [272]:
df_sorted_topo["Model"].value_counts()

Model
Neural Networks    4
Conv1D             1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [273]:
df_sorted_topo_models_grouped = df_sorted_topo_models_cols.groupby("Model").describe().T

<IPython.core.display.Javascript object>

In [274]:
# pd.set_option("display.max_rows", None)
# df_sorted_topo_models_grouped

<IPython.core.display.Javascript object>

In [275]:
df_sorted_topo_models_cols = df_sorted_topo[
    [
        "Model",
        "Dominated_Count",
        "Dominates_Count",
        "Classification",
    ]
]

<IPython.core.display.Javascript object>

In [276]:
(
    df_sorted_topo_models_cols.groupby("Model")[["Classification"]]
    .value_counts(normalize=True)
    .to_frame()
    * 100
).round(2)

proportion
Model           Classification                
Conv1D          intermediate_model       100.0
Neural Networks dominant_model            50.0
                intermediate_model        25.0
                non_dominant_model        25.0

<IPython.core.display.Javascript object>

In [277]:
df_sorted_topo_models_cols.groupby("Model").describe().round(2).T

Model                  Conv1D  Neural Networks
Dominated_Count count     1.0             4.00
                mean      3.0             1.25
                std       NaN             1.89
                min       3.0             0.00
                25%       3.0             0.00
                50%       3.0             0.50
                75%       3.0             1.75
                max       3.0             4.00
Dominates_Count count     1.0             4.00
                mean      1.0             1.75
                std       NaN             1.26
                min       1.0             0.00
                25%       1.0             1.50
                50%       1.0             2.00
                75%       1.0             2.25
                max       1.0             3.00

<IPython.core.display.Javascript object>

In [278]:
# df_sorted_topo_models_cols

<IPython.core.display.Javascript object>

In [279]:
df_sorted_topo_models_cols.groupby("Model")["Classification"].describe()

,count,unique,top,freq
Model,,,,
Conv1D,1,1,intermediate_model,1
Neural Networks,4,3,dominant_model,2


<IPython.core.display.Javascript object>

In [280]:
df_sorted_topo_models_cols.groupby("Model")[["Classification"]].value_counts()

Model            Classification    
Conv1D           intermediate_model    1
Neural Networks  dominant_model        2
                 intermediate_model    1
                 non_dominant_model    1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [281]:
# df_sorted_topo_models_cols

<IPython.core.display.Javascript object>

In [282]:
summary_stats = df_sorted_topo_models_cols.groupby("Model").agg(
    Total_Models=("Model", "count"),
    Mean_Dominance_Count=("Dominates_Count", "mean"),
    Median_Dominance_Count=("Dominates_Count", "median"),
    Total_Dominated=("Dominated_Count", "sum"),
    Total_Dominating=("Dominates_Count", "sum"),
)

total_models = summary_stats[
    "Total_Models"
].sum()  # Get the total number of models across all categories
summary_stats["Dominance Proportion"] = summary_stats["Total_Dominating"] / total_models

<IPython.core.display.Javascript object>

In [283]:
summary_stats

,Total_Models,Mean_Dominance_Count,Median_Dominance_Count,Total_Dominated,Total_Dominating,Dominance Proportion
Model,,,,,,
Conv1D,1,1.00,1.0,3,1,0.2
Neural Networks,4,1.75,2.0,5,7,1.4


<IPython.core.display.Javascript object>

In [284]:
df_sorted_topo_models_grouped.T

RMSE_mean                                                    \
                    count      mean       std       min       25%       50%   
Model                                                                         
Conv1D                1.0  2.289793       NaN  2.289793  2.289793  2.289793   
Neural Networks       4.0  2.385606  0.682431  1.987151  2.029558  2.074281   

                                    RMSE_std            ... Dominated_Count  \
                      75%       max    count      mean  ...             75%   
Model                                                   ...                   
Conv1D           2.289793  2.289793      1.0  0.543307  ...            3.00   
Neural Networks  2.430328  3.406710      4.0  0.802735  ...            1.75   

                     Dominates_Count                                            
                 max           count  mean       std  min  25%  50%   75%  max  
Model                                                                           
Conv1D           3.0             1.0  1.00       NaN  1.0  1.0  1.0  1.00  1.0  
Neural Networks  4.0             4.0  1.75  1.258306  0.0  1.5  2.0  2.25  3.0  

[2 rows x 88 columns]

<IPython.core.display.Javascript object>

### SCPM Analysis

In [285]:
cols = [
    # "Model",
    # "Model_bkp",
    "Model_bkp_2",
    "RMSE_mean",
    "RMSE_std",
    "MAE_mean",
    "MAE_std",
    "MAPE_mean",
    "MAPE_std",
    "R2_mean",
    "R2_std",
    "SCPM",
    "Dominated_Count",
    "Dominates_Count",
    "Classification",
    "Plant",
]

<IPython.core.display.Javascript object>

In [286]:
df_sorted_scpm = df_sorted_topo.sort_values(by="SCPM").copy()

<IPython.core.display.Javascript object>

In [287]:
# (df_sorted_topo[cols].reset_index() == df_sorted_scpm[cols].reset_index()).all(axis=1)
df_sorted_scpm["topo_scmp_order_eq"] = df_sorted_topo.index == df_sorted_scpm.index

<IPython.core.display.Javascript object>

In [288]:
df_sorted_scpm[cols + ["topo_scmp_order_eq"]].round(2).head(5)

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Dominates_Count,Classification,Plant,topo_scmp_order_eq
4,Transformer_1.0,1.99,0.54,1.54,0.38,3.56,1.11,-0.12,1.13,-2.81,0,3,dominant_model,b,False
0,MLP,2.04,0.83,1.60,0.63,3.72,1.77,-0.18,1.10,-2.34,1,2,intermediate_model,b,False
1,BidirectionalLSTM_1.0,2.10,0.90,1.64,0.72,3.84,2.03,-0.05,0.87,-2.08,0,2,dominant_model,b,False
3,Conv1D_7.0,2.29,0.54,1.83,0.44,4.22,1.25,-0.34,1.22,-0.62,3,1,intermediate_model,b,True
2,LSTM_1.0,3.41,0.94,2.68,0.95,5.98,1.88,-3.31,5.94,7.84,4,0,non_dominant_model,b,True


<IPython.core.display.Javascript object>

### Time Series Split

In [289]:
df_results_cm_p = (
    df_results[
        df_results["Features"].eq("Chemical + Mineralogical + Physical - Early CS")
    ]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [290]:
df_results_cm_p["Features"].unique()

array(['Chemical + Mineralogical + Physical - Early CS'], dtype=object)

<IPython.core.display.Javascript object>

In [291]:
df_results_cm_p.shape

(10, 17)

<IPython.core.display.Javascript object>

In [292]:
df_results_cm["Cross Validation"].unique()

array(['Blocking Time Series Split', 'Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [293]:
df_results_cm_p_tss = (
    df_results_cm_p[df_results_cm_p["Cross Validation"].eq("Time Series Split")]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [294]:
df_results_cm_p_tss["Cross Validation"].unique()

array(['Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [295]:
df_results_cm_p_tss.shape

(5, 17)

<IPython.core.display.Javascript object>

###### SCPM computation

In [296]:
df_results_cm_p_tss = compute_scpm(df_results_cm_p_tss)

<IPython.core.display.Javascript object>

##### Dominance Analysis

In [297]:
# Per project
dominance_dict = make_dominance_analysis(df_results_cm_p_tss)
dominance_matrix_cm_p_tss = dominance_dict["dominance_matrix"]
dominance_graph_cm_p_tss = dominance_dict["dominance_graph"]
df_sorted_count = dominance_dict["df_sorted_count"]
df_sorted_topo = dominance_dict["df_sorted_topo"]

<IPython.core.display.Javascript object>

In [298]:
dominance_matrix_cm_tss.shape, len(dominance_graph_cm_tss)

((5, 5), 5)

<IPython.core.display.Javascript object>

In [299]:
df_sorted_topo["Classification"].value_counts()

Classification
intermediate_model    3
dominant_model        1
non_dominant_model    1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [300]:
df_sorted_topo.to_csv(
    "../../../../reports/results/global_models/ecics/ecics_206_b_dominance_analysis_cm_p_tss.csv",
    index=False,
)

<IPython.core.display.Javascript object>

In [301]:
df_sorted_topo.head(5)

,Company,Plant,Cross Validation,Features,Features_bkp,Model,Model_bkp,Model_bkp_2,Timesteps,RMSE_mean,...,MAPE_mean,R2_mean,RMSE_std,MAE_std,MAPE_std,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count
0,206,b,Time Series Split,Chemical + Mineralogical + Physical - Early CS,Chemical + Properties CS Less,Neural Networks,MLP,MLP,NaN,1.858427,...,3.203862,0.497233,0.417326,0.298023,0.737698,0.120027,-3.889748,0,dominant_model,4
4,206,b,Time Series Split,Chemical + Mineralogical + Physical - Early CS,Chemical + Properties CS Less,Neural Networks,Transformer,Transformer_1.0,1.0,1.887594,...,3.282975,0.480595,0.473380,0.348747,0.863810,0.143988,-2.609870,1,intermediate_model,3
3,206,b,Time Series Split,Chemical + Mineralogical + Physical - Early CS,Chemical + Properties CS Less,Conv1D,Conv1D,Conv1D_7.0,7.0,1.896894,...,3.324535,0.475589,0.497745,0.388630,0.962450,0.126703,-1.990905,2,intermediate_model,2
1,206,b,Time Series Split,Chemical + Mineralogical + Physical - Early CS,Chemical + Properties CS Less,Neural Networks,BidirectionalLSTM,BidirectionalLSTM_1.0,1.0,2.021431,...,3.455559,0.392963,0.390474,0.240811,0.639323,0.163484,1.281032,3,intermediate_model,1
2,206,b,Time Series Split,Chemical + Mineralogical + Physical - Early CS,Chemical + Properties CS Less,Neural Networks,LSTM,LSTM_1.0,1.0,2.256970,...,3.687364,0.253518,0.507799,0.357134,0.827348,0.221712,7.209492,4,non_dominant_model,0


<IPython.core.display.Javascript object>

In [302]:
df_sorted_topo_cols = df_sorted_topo[
    [
        # "Model",
        # "Model_bkp",
        "Model_bkp_2",
        "RMSE_mean",
        "RMSE_std",
        "MAE_mean",
        "MAE_std",
        "MAPE_mean",
        "MAPE_std",
        "R2_mean",
        "R2_std",
        "SCPM",
        "Dominated_Count",
        "Classification",
        "Dominates_Count",
        "Plant",
    ]
]  # .round(2)

<IPython.core.display.Javascript object>

###### Dominant Models

In [303]:
# Top 4
df_sorted_topo_cols.sort_values(
    by=["Dominates_Count", "SCPM"], ascending=[False, True]
).iloc[:2].round(2)

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
0,MLP,1.86,0.42,1.42,0.30,3.20,0.74,0.50,0.12,-3.89,0,dominant_model,4,b
4,Transformer_1.0,1.89,0.47,1.45,0.35,3.28,0.86,0.48,0.14,-2.61,1,intermediate_model,3,b


<IPython.core.display.Javascript object>

In [304]:
df_sorted_topo_cols[df_sorted_topo_cols["Classification"].eq("dominant_model")].shape

(1, 14)

<IPython.core.display.Javascript object>

###### Intermediate models

In [305]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("intermediate_model")
].sort_values(by=["Dominates_Count", "SCPM"], ascending=[False, True]).iloc[:4].round(2)

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
4,Transformer_1.0,1.89,0.47,1.45,0.35,3.28,0.86,0.48,0.14,-2.61,1,intermediate_model,3,b
3,Conv1D_7.0,1.90,0.50,1.47,0.39,3.32,0.96,0.48,0.13,-1.99,2,intermediate_model,2,b
1,BidirectionalLSTM_1.0,2.02,0.39,1.53,0.24,3.46,0.64,0.39,0.16,1.28,3,intermediate_model,1,b


<IPython.core.display.Javascript object>

In [306]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("intermediate_model")
].shape

(3, 14)

<IPython.core.display.Javascript object>

###### non-dominant models

In [307]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("non_dominant_model")
].sort_values(by=["Dominates_Count", "SCPM"], ascending=[False, True]).iloc[:3]

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
2,LSTM_1.0,2.25697,0.507799,1.637938,0.357134,3.687364,0.827348,0.253518,0.221712,7.209492,4,non_dominant_model,0,b


<IPython.core.display.Javascript object>

In [308]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("non_dominant_model")
].shape

(1, 14)

<IPython.core.display.Javascript object>

###### Statistics per model type

In [309]:
df_sorted_topo_models_cols = df_sorted_topo[
    [
        "Model",
        # "Model_bkp",
        # "Model_bkp_2",
        "RMSE_mean",
        "RMSE_std",
        "MAE_mean",
        "MAE_std",
        "MAPE_mean",
        "MAPE_std",
        "R2_mean",
        "R2_std",
        "SCPM",
        "Dominated_Count",
        "Dominates_Count",
        "Classification",
        "Plant",
    ]
]  # .round(2)

<IPython.core.display.Javascript object>

In [310]:
df_sorted_topo["Model"].value_counts()

Model
Neural Networks    4
Conv1D             1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [311]:
df_sorted_topo_models_grouped = df_sorted_topo_models_cols.groupby("Model").describe().T

<IPython.core.display.Javascript object>

In [312]:
df_sorted_topo_models_cols = df_sorted_topo[
    [
        "Model",
        "Dominated_Count",
        "Dominates_Count",
        "Classification",
    ]
]

<IPython.core.display.Javascript object>

In [313]:
(
    df_sorted_topo_models_cols.groupby("Model")[["Classification"]]
    .value_counts(normalize=True)
    .to_frame()
    * 100
).round(2)

proportion
Model           Classification                
Conv1D          intermediate_model       100.0
Neural Networks intermediate_model        50.0
                dominant_model            25.0
                non_dominant_model        25.0

<IPython.core.display.Javascript object>

In [314]:
df_sorted_topo_models_cols.groupby("Model").describe().round(2).T

Model                  Conv1D  Neural Networks
Dominated_Count count     1.0             4.00
                mean      2.0             2.00
                std       NaN             1.83
                min       2.0             0.00
                25%       2.0             0.75
                50%       2.0             2.00
                75%       2.0             3.25
                max       2.0             4.00
Dominates_Count count     1.0             4.00
                mean      2.0             2.00
                std       NaN             1.83
                min       2.0             0.00
                25%       2.0             0.75
                50%       2.0             2.00
                75%       2.0             3.25
                max       2.0             4.00

<IPython.core.display.Javascript object>

In [315]:
# df_sorted_topo_models_cols

<IPython.core.display.Javascript object>

In [316]:
df_sorted_topo_models_cols.groupby("Model")["Classification"].describe()

,count,unique,top,freq
Model,,,,
Conv1D,1,1,intermediate_model,1
Neural Networks,4,3,intermediate_model,2


<IPython.core.display.Javascript object>

In [317]:
df_sorted_topo_models_cols.groupby("Model")[["Classification"]].value_counts()

Model            Classification    
Conv1D           intermediate_model    1
Neural Networks  intermediate_model    2
                 dominant_model        1
                 non_dominant_model    1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [318]:
summary_stats = df_sorted_topo_models_cols.groupby("Model").agg(
    Total_Models=("Model", "count"),
    Mean_Dominance_Count=("Dominates_Count", "mean"),
    Median_Dominance_Count=("Dominates_Count", "median"),
    Total_Dominated=("Dominated_Count", "sum"),
    Total_Dominating=("Dominates_Count", "sum"),
)

total_models = summary_stats[
    "Total_Models"
].sum()  # Get the total number of models across all categories
summary_stats["Dominance Proportion"] = summary_stats["Total_Dominating"] / total_models

<IPython.core.display.Javascript object>

In [319]:
summary_stats.round(2)

,Total_Models,Mean_Dominance_Count,Median_Dominance_Count,Total_Dominated,Total_Dominating,Dominance Proportion
Model,,,,,,
Conv1D,1,2.0,2.0,2,2,0.4
Neural Networks,4,2.0,2.0,8,8,1.6


<IPython.core.display.Javascript object>

In [320]:
df_sorted_topo_models_grouped.T

RMSE_mean                                                    \
                    count      mean       std       min       25%       50%   
Model                                                                         
Conv1D                1.0  1.896894       NaN  1.896894  1.896894  1.896894   
Neural Networks       4.0  2.006105  0.181679  1.858427  1.880302  1.954512   

                                    RMSE_std            ... Dominated_Count  \
                      75%       max    count      mean  ...             75%   
Model                                                   ...                   
Conv1D           1.896894  1.896894      1.0  0.497745  ...            2.00   
Neural Networks  2.080315  2.256970      4.0  0.447245  ...            3.25   

                     Dominates_Count                                            
                 max           count mean       std  min   25%  50%   75%  max  
Model                                                                           
Conv1D           2.0             1.0  2.0       NaN  2.0  2.00  2.0  2.00  2.0  
Neural Networks  4.0             4.0  2.0  1.825742  0.0  0.75  2.0  3.25  4.0  

[2 rows x 88 columns]

<IPython.core.display.Javascript object>

### SCPM Analysis

In [321]:
cols = [
    # "Model",
    # "Model_bkp",
    "Model_bkp_2",
    "RMSE_mean",
    "RMSE_std",
    "MAE_mean",
    "MAE_std",
    "MAPE_mean",
    "MAPE_std",
    "R2_mean",
    "R2_std",
    "SCPM",
    "Dominated_Count",
    "Dominates_Count",
    "Classification",
    "Plant",
]

<IPython.core.display.Javascript object>

#### SCPM Analysis by Project

In [322]:
df_sorted_scpm = df_sorted_topo.sort_values(by="SCPM").copy()

<IPython.core.display.Javascript object>

In [323]:
# (df_sorted_topo[cols].reset_index() == df_sorted_scpm[cols].reset_index()).all(axis=1)
df_sorted_scpm["topo_scmp_order_eq"] = df_sorted_topo.index == df_sorted_scpm.index

<IPython.core.display.Javascript object>

In [324]:
df_sorted_scpm[cols + ["topo_scmp_order_eq"]].round(2).head(5)

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Dominates_Count,Classification,Plant,topo_scmp_order_eq
0,MLP,1.86,0.42,1.42,0.30,3.20,0.74,0.50,0.12,-3.89,0,4,dominant_model,b,True
4,Transformer_1.0,1.89,0.47,1.45,0.35,3.28,0.86,0.48,0.14,-2.61,1,3,intermediate_model,b,True
3,Conv1D_7.0,1.90,0.50,1.47,0.39,3.32,0.96,0.48,0.13,-1.99,2,2,intermediate_model,b,True
1,BidirectionalLSTM_1.0,2.02,0.39,1.53,0.24,3.46,0.64,0.39,0.16,1.28,3,1,intermediate_model,b,True
2,LSTM_1.0,2.26,0.51,1.64,0.36,3.69,0.83,0.25,0.22,7.21,4,0,non_dominant_model,b,True


<IPython.core.display.Javascript object>

## Variable Grouping: CM-P-CS

#### Analysis of the best models identified in the Local Analysis

**Best models for variable groupping CM-P-CS were from:**

*BTiSS:*
    Dominance analysis: Plant S and Plant B
    SCPM: Plant S and Plant B

*TiSS:*
    Dominance analysis: Plant AB
    SCPM: Plant AB

In [325]:
df_results["Features"].unique()

array(['Chemical + Mineralogical', 'Chemical + Mineralogical + Physical',
       'Chemical + Mineralogical + Physical - Early CS'], dtype=object)

<IPython.core.display.Javascript object>

### Blocking time series

### Plant B

In [326]:
df_results_cm_p_cs = (
    df_results[df_results["Features"].eq("Chemical + Mineralogical + Physical")]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [327]:
df_results_cm_p_cs["Features"].unique()

array(['Chemical + Mineralogical + Physical'], dtype=object)

<IPython.core.display.Javascript object>

In [328]:
df_results_cm_p_cs.shape

(10, 17)

<IPython.core.display.Javascript object>

In [329]:
df_results_cm_p_cs = df_results_cm_p_cs[
    df_results_cm_p_cs["Plant"].eq("b")
].reset_index(drop=True)

<IPython.core.display.Javascript object>

In [330]:
df_results_cm_p_cs["Cross Validation"].unique()

array(['Blocking Time Series Split', 'Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [331]:
df_results_cm_p_cs_btss = (
    df_results_cm_p_cs[
        df_results_cm_p_cs["Cross Validation"].eq("Blocking Time Series Split")
    ]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [332]:
df_results_cm_p_cs_btss["Cross Validation"].unique()

array(['Blocking Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [333]:
df_results_cm_p_cs_btss.shape

(5, 17)

<IPython.core.display.Javascript object>

###### SCPM computation

In [334]:
df_results_cm_p_cs_btss = compute_scpm(df_results_cm_p_cs_btss)

<IPython.core.display.Javascript object>

##### Dominance Analysis

In [335]:
# Per project
dominance_dict = make_dominance_analysis(df_results_cm_p_cs_btss)
dominance_matrix_cm_p_cs_btss = dominance_dict["dominance_matrix"]
dominance_graph_cm_p_cs_btss = dominance_dict["dominance_graph"]
df_sorted_count = dominance_dict["df_sorted_count"]
df_sorted_topo = dominance_dict["df_sorted_topo"]

<IPython.core.display.Javascript object>

In [336]:
dominance_matrix_cm_p_cs_btss.shape, len(dominance_graph_cm_p_cs_btss)

((5, 5), 5)

<IPython.core.display.Javascript object>

In [337]:
df_sorted_topo["Classification"].value_counts()

Classification
dominant_model        4
non_dominant_model    1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [338]:
df_sorted_topo.to_csv(
    "../../../../reports/results/global_models/ecics/ecics_206_b_dominance_analysis_cm_p_cs_btss.csv",
    index=False,
)

<IPython.core.display.Javascript object>

In [339]:
df_sorted_topo.head(5)

,Company,Plant,Cross Validation,Features,Features_bkp,Model,Model_bkp,Model_bkp_2,Timesteps,RMSE_mean,...,MAPE_mean,R2_mean,RMSE_std,MAE_std,MAPE_std,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count
0,206,b,Blocking Time Series Split,Chemical + Mineralogical + Physical,Chemical + Physical,Neural Networks,MLP,MLP,NaN,1.947471,...,3.122615,-0.025926,0.714070,0.382674,0.811913,1.115282,-2.531426,0,dominant_model,1
2,206,b,Blocking Time Series Split,Chemical + Mineralogical + Physical,Chemical + Physical,Neural Networks,Transformer,Transformer_1.0,1.0,1.834307,...,3.224629,-0.103292,0.558644,0.392689,1.035626,1.314121,-2.742737,0,dominant_model,1
3,206,b,Blocking Time Series Split,Chemical + Mineralogical + Physical,Chemical + Physical,Neural Networks,LSTM,LSTM_7.0,7.0,2.016623,...,3.750301,0.133919,0.606551,0.581108,1.501075,0.559036,0.214189,0,dominant_model,1
4,206,b,Blocking Time Series Split,Chemical + Mineralogical + Physical,Chemical + Physical,Neural Networks,BidirectionalLSTM,BidirectionalLSTM_1.0,1.0,1.863905,...,3.113497,-0.559881,0.692567,0.435946,0.946320,2.346148,-2.051518,0,dominant_model,1
1,206,b,Blocking Time Series Split,Chemical + Mineralogical + Physical,Chemical + Physical,Conv1D,Conv1D,Conv1D_1.0,1.0,2.329108,...,4.283906,-1.314728,0.838553,0.727417,1.604814,3.388252,7.111492,4,non_dominant_model,0


<IPython.core.display.Javascript object>

In [340]:
df_sorted_topo_cols = df_sorted_topo[
    [
        # "Model",
        # "Model_bkp",
        "Model_bkp_2",
        "RMSE_mean",
        "RMSE_std",
        "MAE_mean",
        "MAE_std",
        "MAPE_mean",
        "MAPE_std",
        "R2_mean",
        "R2_std",
        "SCPM",
        "Dominated_Count",
        "Classification",
        "Dominates_Count",
        "Plant",
    ]
].round(2)

<IPython.core.display.Javascript object>

###### Dominant Models

In [341]:
# Top 4
df_sorted_topo_cols.sort_values(
    by=["Dominates_Count", "SCPM"], ascending=[False, True]
).iloc[:5]

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
2,Transformer_1.0,1.83,0.56,1.40,0.39,3.22,1.04,-0.10,1.31,-2.74,0,dominant_model,1,b
0,MLP,1.95,0.71,1.39,0.38,3.12,0.81,-0.03,1.12,-2.53,0,dominant_model,1,b
4,BidirectionalLSTM_1.0,1.86,0.69,1.38,0.44,3.11,0.95,-0.56,2.35,-2.05,0,dominant_model,1,b
3,LSTM_7.0,2.02,0.61,1.64,0.58,3.75,1.50,0.13,0.56,0.21,0,dominant_model,1,b
1,Conv1D_1.0,2.33,0.84,1.88,0.73,4.28,1.60,-1.31,3.39,7.11,4,non_dominant_model,0,b


<IPython.core.display.Javascript object>

In [342]:
df_sorted_topo_cols[df_sorted_topo_cols["Classification"].eq("dominant_model")].shape

(4, 14)

<IPython.core.display.Javascript object>

###### Intermediate models

In [343]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("intermediate_model")
].sort_values(by=["Dominates_Count", "SCPM"], ascending=[False, True]).iloc[:3]

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count,Plant


<IPython.core.display.Javascript object>

In [344]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("intermediate_model")
].shape

(0, 14)

<IPython.core.display.Javascript object>

###### non-dominant models

In [345]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("non_dominant_model")
].sort_values(by=["Dominates_Count", "SCPM"], ascending=[False, True]).iloc[:2]

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
1,Conv1D_1.0,2.33,0.84,1.88,0.73,4.28,1.6,-1.31,3.39,7.11,4,non_dominant_model,0,b


<IPython.core.display.Javascript object>

In [346]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("non_dominant_model")
].shape

(1, 14)

<IPython.core.display.Javascript object>

###### Statistics per model type

In [347]:
df_sorted_topo_models_cols = df_sorted_topo[
    [
        "Model",
        # "Model_bkp",
        # "Model_bkp_2",
        "RMSE_mean",
        "RMSE_std",
        "MAE_mean",
        "MAE_std",
        "MAPE_mean",
        "MAPE_std",
        "R2_mean",
        "R2_std",
        "SCPM",
        "Dominated_Count",
        "Dominates_Count",
        "Classification",
        "Plant",
    ]
]  # .round(2)

<IPython.core.display.Javascript object>

In [348]:
df_sorted_topo["Model"].value_counts()

Model
Neural Networks    4
Conv1D             1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [349]:
df_sorted_topo_models_grouped = df_sorted_topo_models_cols.groupby("Model").describe().T

<IPython.core.display.Javascript object>

In [350]:
# pd.set_option("display.max_rows", None)
# df_sorted_topo_models_grouped

<IPython.core.display.Javascript object>

In [351]:
df_sorted_topo_models_cols = df_sorted_topo[
    [
        "Model",
        "Dominated_Count",
        "Dominates_Count",
        "Classification",
    ]
]

<IPython.core.display.Javascript object>

In [352]:
(
    df_sorted_topo_models_cols.groupby("Model")[["Classification"]]
    .value_counts(normalize=True)
    .to_frame()
    * 100
).round(2)

,,proportion
Model,Classification,
Conv1D,non_dominant_model,100.0
Neural Networks,dominant_model,100.0


<IPython.core.display.Javascript object>

In [353]:
df_sorted_topo_models_cols.groupby("Model").describe().round(2).T

Model                  Conv1D  Neural Networks
Dominated_Count count     1.0              4.0
                mean      4.0              0.0
                std       NaN              0.0
                min       4.0              0.0
                25%       4.0              0.0
                50%       4.0              0.0
                75%       4.0              0.0
                max       4.0              0.0
Dominates_Count count     1.0              4.0
                mean      0.0              1.0
                std       NaN              0.0
                min       0.0              1.0
                25%       0.0              1.0
                50%       0.0              1.0
                75%       0.0              1.0
                max       0.0              1.0

<IPython.core.display.Javascript object>

In [354]:
# df_sorted_topo_models_cols

<IPython.core.display.Javascript object>

In [355]:
df_sorted_topo_models_cols.groupby("Model")["Classification"].describe()

,count,unique,top,freq
Model,,,,
Conv1D,1,1,non_dominant_model,1
Neural Networks,4,1,dominant_model,4


<IPython.core.display.Javascript object>

In [356]:
df_sorted_topo_models_cols.groupby("Model")[["Classification"]].value_counts()

Model            Classification    
Conv1D           non_dominant_model    1
Neural Networks  dominant_model        4
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [357]:
# df_sorted_topo_models_cols

<IPython.core.display.Javascript object>

In [358]:
summary_stats = df_sorted_topo_models_cols.groupby("Model").agg(
    Total_Models=("Model", "count"),
    Mean_Dominance_Count=("Dominates_Count", "mean"),
    Median_Dominance_Count=("Dominates_Count", "median"),
    Total_Dominated=("Dominated_Count", "sum"),
    Total_Dominating=("Dominates_Count", "sum"),
)

total_models = summary_stats[
    "Total_Models"
].sum()  # Get the total number of models across all categories
summary_stats["Dominance Proportion"] = summary_stats["Total_Dominating"] / total_models

<IPython.core.display.Javascript object>

In [359]:
summary_stats

,Total_Models,Mean_Dominance_Count,Median_Dominance_Count,Total_Dominated,Total_Dominating,Dominance Proportion
Model,,,,,,
Conv1D,1,0.0,0.0,4,0,0.0
Neural Networks,4,1.0,1.0,0,4,0.8


<IPython.core.display.Javascript object>

In [360]:
df_sorted_topo_models_grouped.T

RMSE_mean                                                    \
                    count      mean       std       min       25%       50%   
Model                                                                         
Conv1D                1.0  2.329108       NaN  2.329108  2.329108  2.329108   
Neural Networks       4.0  1.915577  0.082669  1.834307  1.856506  1.905688   

                                    RMSE_std            ... Dominated_Count  \
                      75%       max    count      mean  ...             75%   
Model                                                   ...                   
Conv1D           2.329108  2.329108      1.0  0.838553  ...             4.0   
Neural Networks  1.964759  2.016623      4.0  0.642958  ...             0.0   

                     Dominates_Count                                     
                 max           count mean  std  min  25%  50%  75%  max  
Model                                                                    
Conv1D           4.0             1.0  0.0  NaN  0.0  0.0  0.0  0.0  0.0  
Neural Networks  0.0             4.0  1.0  0.0  1.0  1.0  1.0  1.0  1.0  

[2 rows x 88 columns]

<IPython.core.display.Javascript object>

### SCPM Analysis

In [361]:
cols = [
    # "Model",
    # "Model_bkp",
    "Model_bkp_2",
    "RMSE_mean",
    "RMSE_std",
    "MAE_mean",
    "MAE_std",
    "MAPE_mean",
    "MAPE_std",
    "R2_mean",
    "R2_std",
    "SCPM",
    "Dominated_Count",
    "Dominates_Count",
    "Classification",
    "Plant",
]

<IPython.core.display.Javascript object>

#### SCPM Analysis by Project

In [362]:
df_sorted_scpm = df_sorted_topo.sort_values(by="SCPM").copy()

<IPython.core.display.Javascript object>

In [363]:
# (df_sorted_topo[cols].reset_index() == df_sorted_scpm[cols].reset_index()).all(axis=1)
df_sorted_scpm["topo_scmp_order_eq"] = df_sorted_topo.index == df_sorted_scpm.index

<IPython.core.display.Javascript object>

In [364]:
df_sorted_scpm[cols + ["topo_scmp_order_eq"]].round(2).head(5)

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Dominates_Count,Classification,Plant,topo_scmp_order_eq
2,Transformer_1.0,1.83,0.56,1.40,0.39,3.22,1.04,-0.10,1.31,-2.74,0,1,dominant_model,b,False
0,MLP,1.95,0.71,1.39,0.38,3.12,0.81,-0.03,1.12,-2.53,0,1,dominant_model,b,False
4,BidirectionalLSTM_1.0,1.86,0.69,1.38,0.44,3.11,0.95,-0.56,2.35,-2.05,0,1,dominant_model,b,False
3,LSTM_7.0,2.02,0.61,1.64,0.58,3.75,1.50,0.13,0.56,0.21,0,1,dominant_model,b,False
1,Conv1D_1.0,2.33,0.84,1.88,0.73,4.28,1.60,-1.31,3.39,7.11,4,0,non_dominant_model,b,True


<IPython.core.display.Javascript object>

### Time Series Split

In [365]:
df_results_cm_p_cs = (
    df_results[df_results["Features"].eq("Chemical + Mineralogical + Physical")]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [366]:
df_results_cm_p_cs["Features"].unique()

array(['Chemical + Mineralogical + Physical'], dtype=object)

<IPython.core.display.Javascript object>

In [367]:
df_results_cm_p_cs.shape

(10, 17)

<IPython.core.display.Javascript object>

### Plant B

In [368]:
df_results_cm_p_cs = df_results_cm_p_cs[
    df_results_cm_p_cs["Plant"].eq("b")
].reset_index(drop=True)

<IPython.core.display.Javascript object>

In [369]:
df_results_cm_p_cs["Cross Validation"].unique()

array(['Blocking Time Series Split', 'Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [370]:
df_results_cm_p_cs_tss = (
    df_results_cm_p_cs[df_results_cm_p_cs["Cross Validation"].eq("Time Series Split")]
    .copy()
    .reset_index(drop=True)
)

<IPython.core.display.Javascript object>

In [371]:
df_results_cm_p_cs_tss["Cross Validation"].unique()

array(['Time Series Split'], dtype=object)

<IPython.core.display.Javascript object>

In [372]:
df_results_cm_p_cs_tss.shape

(5, 17)

<IPython.core.display.Javascript object>

###### SCPM computation

In [373]:
df_results_cm_p_cs_tss = compute_scpm(df_results_cm_p_cs_tss)

<IPython.core.display.Javascript object>

##### Dominance Analysis

In [374]:
# Per project
dominance_dict = make_dominance_analysis(df_results_cm_p_cs_tss)
dominance_matrix_cm_p_cs_tss = dominance_dict["dominance_matrix"]
dominance_graph_cm_p_cs_tss = dominance_dict["dominance_graph"]
df_sorted_count = dominance_dict["df_sorted_count"]
df_sorted_topo = dominance_dict["df_sorted_topo"]

<IPython.core.display.Javascript object>

In [375]:
dominance_matrix_cm_p_cs_tss.shape, len(dominance_graph_cm_p_cs_tss)

((5, 5), 5)

<IPython.core.display.Javascript object>

In [376]:
df_sorted_topo["Classification"].value_counts()

Classification
intermediate_model    2
non_dominant_model    2
dominant_model        1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [377]:
df_sorted_topo.to_csv(
    "../../../../reports/results/global_models/ecics/ecics_206_b_dominance_analysis_cm_p_cs_tss.csv",
    index=False,
)

<IPython.core.display.Javascript object>

In [378]:
df_sorted_topo.head(5)

,Company,Plant,Cross Validation,Features,Features_bkp,Model,Model_bkp,Model_bkp_2,Timesteps,RMSE_mean,...,MAPE_mean,R2_mean,RMSE_std,MAE_std,MAPE_std,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count
4,206,b,Time Series Split,Chemical + Mineralogical + Physical,Chemical + Physical,Neural Networks,BidirectionalLSTM,BidirectionalLSTM_1.0,1.0,1.534663,...,2.618739,0.651091,0.237837,0.162978,0.398202,0.076946,-5.358671,0,dominant_model,4
1,206,b,Time Series Split,Chemical + Mineralogical + Physical,Chemical + Physical,Conv1D,Conv1D,Conv1D_1.0,1.0,1.714384,...,2.988149,0.564217,0.444534,0.364359,0.812482,0.162360,-0.936400,1,intermediate_model,2
2,206,b,Time Series Split,Chemical + Mineralogical + Physical,Chemical + Physical,Neural Networks,Transformer,Transformer_1.0,1.0,1.722620,...,2.898967,0.561525,0.255570,0.242436,0.566274,0.087192,-1.642500,1,intermediate_model,2
0,206,b,Time Series Split,Chemical + Mineralogical + Physical,Chemical + Physical,Neural Networks,MLP,MLP,NaN,2.098765,...,2.994923,0.252014,0.806614,0.262669,0.531123,0.546545,3.439019,3,non_dominant_model,0
3,206,b,Time Series Split,Chemical + Mineralogical + Physical,Chemical + Physical,Neural Networks,LSTM,LSTM_7.0,7.0,1.939356,...,3.419520,0.430404,0.349961,0.265969,0.663518,0.146204,4.498553,3,non_dominant_model,0


<IPython.core.display.Javascript object>

In [379]:
df_sorted_topo_cols = df_sorted_topo[
    [
        # "Model",
        # "Model_bkp",
        "Model_bkp_2",
        "RMSE_mean",
        "RMSE_std",
        "MAE_mean",
        "MAE_std",
        "MAPE_mean",
        "MAPE_std",
        "R2_mean",
        "R2_std",
        "SCPM",
        "Dominated_Count",
        "Classification",
        "Dominates_Count",
        "Plant",
    ]
]  # .round(2)

<IPython.core.display.Javascript object>

###### Dominant Models

In [380]:
# Top 4
df_sorted_topo_cols.iloc[0:2].sort_values(
    by=["Dominates_Count", "SCPM"], ascending=[False, True]
).round(2)

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
4,BidirectionalLSTM_1.0,1.53,0.24,1.17,0.16,2.62,0.40,0.65,0.08,-5.36,0,dominant_model,4,b
1,Conv1D_1.0,1.71,0.44,1.34,0.36,2.99,0.81,0.56,0.16,-0.94,1,intermediate_model,2,b


<IPython.core.display.Javascript object>

In [381]:
df_sorted_topo_cols[df_sorted_topo_cols["Classification"].eq("dominant_model")].shape

(1, 14)

<IPython.core.display.Javascript object>

###### Intermediate models

In [382]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("intermediate_model")
].sort_values(by=["Dominates_Count", "SCPM"], ascending=[False, True]).iloc[:4].round(2)

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
2,Transformer_1.0,1.72,0.26,1.29,0.24,2.90,0.57,0.56,0.09,-1.64,1,intermediate_model,2,b
1,Conv1D_1.0,1.71,0.44,1.34,0.36,2.99,0.81,0.56,0.16,-0.94,1,intermediate_model,2,b


<IPython.core.display.Javascript object>

In [383]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("intermediate_model")
].shape

(2, 14)

<IPython.core.display.Javascript object>

###### non-dominant models

In [384]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("non_dominant_model")
].sort_values(by=["Dominates_Count", "SCPM"], ascending=[False, True]).iloc[:3]

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Classification,Dominates_Count,Plant
0,MLP,2.098765,0.806614,1.355404,0.262669,2.994923,0.531123,0.252014,0.546545,3.439019,3,non_dominant_model,0,b
3,LSTM_7.0,1.939356,0.349961,1.528852,0.265969,3.419520,0.663518,0.430404,0.146204,4.498553,3,non_dominant_model,0,b


<IPython.core.display.Javascript object>

In [385]:
df_sorted_topo_cols[
    df_sorted_topo_cols["Classification"].eq("non_dominant_model")
].shape

(2, 14)

<IPython.core.display.Javascript object>

###### Statistics per model type

In [386]:
df_sorted_topo_models_cols = df_sorted_topo[
    [
        "Model",
        # "Model_bkp",
        # "Model_bkp_2",
        "RMSE_mean",
        "RMSE_std",
        "MAE_mean",
        "MAE_std",
        "MAPE_mean",
        "MAPE_std",
        "R2_mean",
        "R2_std",
        "SCPM",
        "Dominated_Count",
        "Dominates_Count",
        "Classification",
        "Plant",
    ]
]  # .round(2)

<IPython.core.display.Javascript object>

In [387]:
df_sorted_topo["Model"].value_counts()

Model
Neural Networks    4
Conv1D             1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [388]:
df_sorted_topo_models_grouped = df_sorted_topo_models_cols.groupby("Model").describe().T

<IPython.core.display.Javascript object>

In [389]:
df_sorted_topo_models_cols = df_sorted_topo[
    [
        "Model",
        "Dominated_Count",
        "Dominates_Count",
        "Classification",
    ]
]

<IPython.core.display.Javascript object>

In [390]:
(
    df_sorted_topo_models_cols.groupby("Model")[["Classification"]]
    .value_counts(normalize=True)
    .to_frame()
    * 100
).round(2)

proportion
Model           Classification                
Conv1D          intermediate_model       100.0
Neural Networks non_dominant_model        50.0
                dominant_model            25.0
                intermediate_model        25.0

<IPython.core.display.Javascript object>

In [391]:
df_sorted_topo_models_cols.groupby("Model").describe().round(2).T

Model                  Conv1D  Neural Networks
Dominated_Count count     1.0             4.00
                mean      1.0             1.75
                std       NaN             1.50
                min       1.0             0.00
                25%       1.0             0.75
                50%       1.0             2.00
                75%       1.0             3.00
                max       1.0             3.00
Dominates_Count count     1.0             4.00
                mean      2.0             1.50
                std       NaN             1.91
                min       2.0             0.00
                25%       2.0             0.00
                50%       2.0             1.00
                75%       2.0             2.50
                max       2.0             4.00

<IPython.core.display.Javascript object>

In [392]:
# df_sorted_topo_models_cols

<IPython.core.display.Javascript object>

In [393]:
df_sorted_topo_models_cols.groupby("Model")["Classification"].describe()

,count,unique,top,freq
Model,,,,
Conv1D,1,1,intermediate_model,1
Neural Networks,4,3,non_dominant_model,2


<IPython.core.display.Javascript object>

In [394]:
df_sorted_topo_models_cols.groupby("Model")[["Classification"]].value_counts()

Model            Classification    
Conv1D           intermediate_model    1
Neural Networks  non_dominant_model    2
                 dominant_model        1
                 intermediate_model    1
Name: count, dtype: int64

<IPython.core.display.Javascript object>

In [395]:
summary_stats = df_sorted_topo_models_cols.groupby("Model").agg(
    Total_Models=("Model", "count"),
    Mean_Dominance_Count=("Dominates_Count", "mean"),
    Median_Dominance_Count=("Dominates_Count", "median"),
    Total_Dominated=("Dominated_Count", "sum"),
    Total_Dominating=("Dominates_Count", "sum"),
)

total_models = summary_stats[
    "Total_Models"
].sum()  # Get the total number of models across all categories
summary_stats["Dominance Proportion"] = summary_stats["Total_Dominating"] / total_models

<IPython.core.display.Javascript object>

In [396]:
summary_stats.round(2)

,Total_Models,Mean_Dominance_Count,Median_Dominance_Count,Total_Dominated,Total_Dominating,Dominance Proportion
Model,,,,,,
Conv1D,1,2.0,2.0,1,2,0.4
Neural Networks,4,1.5,1.0,7,6,1.2


<IPython.core.display.Javascript object>

In [397]:
df_sorted_topo_models_grouped.T

RMSE_mean                                                    \
                    count      mean       std       min       25%       50%   
Model                                                                         
Conv1D                1.0  1.714384       NaN  1.714384  1.714384  1.714384   
Neural Networks       4.0  1.823851  0.246844  1.534663  1.675631  1.830988   

                                    RMSE_std            ... Dominated_Count  \
                      75%       max    count      mean  ...             75%   
Model                                                   ...                   
Conv1D           1.714384  1.714384      1.0  0.444534  ...             1.0   
Neural Networks  1.979208  2.098765      4.0  0.412496  ...             3.0   

                     Dominates_Count                                          
                 max           count mean       std  min  25%  50%  75%  max  
Model                                                                         
Conv1D           1.0             1.0  2.0       NaN  2.0  2.0  2.0  2.0  2.0  
Neural Networks  3.0             4.0  1.5  1.914854  0.0  0.0  1.0  2.5  4.0  

[2 rows x 88 columns]

<IPython.core.display.Javascript object>

### SCPM Analysis

In [398]:
cols = [
    # "Model",
    # "Model_bkp",
    "Model_bkp_2",
    "RMSE_mean",
    "RMSE_std",
    "MAE_mean",
    "MAE_std",
    "MAPE_mean",
    "MAPE_std",
    "R2_mean",
    "R2_std",
    "SCPM",
    "Dominated_Count",
    "Dominates_Count",
    "Classification",
    "Plant",
]

<IPython.core.display.Javascript object>

#### SCPM Analysis by Project

In [399]:
df_sorted_scpm = df_sorted_topo.sort_values(by="SCPM").copy()

<IPython.core.display.Javascript object>

In [400]:
# (df_sorted_topo[cols].reset_index() == df_sorted_scpm[cols].reset_index()).all(axis=1)
df_sorted_scpm["topo_scmp_order_eq"] = df_sorted_topo.index == df_sorted_scpm.index

<IPython.core.display.Javascript object>

In [401]:
df_sorted_scpm[cols + ["topo_scmp_order_eq"]].round(2).head(5)

,Model_bkp_2,RMSE_mean,RMSE_std,MAE_mean,MAE_std,MAPE_mean,MAPE_std,R2_mean,R2_std,SCPM,Dominated_Count,Dominates_Count,Classification,Plant,topo_scmp_order_eq
4,BidirectionalLSTM_1.0,1.53,0.24,1.17,0.16,2.62,0.40,0.65,0.08,-5.36,0,4,dominant_model,b,True
2,Transformer_1.0,1.72,0.26,1.29,0.24,2.90,0.57,0.56,0.09,-1.64,1,2,intermediate_model,b,False
1,Conv1D_1.0,1.71,0.44,1.34,0.36,2.99,0.81,0.56,0.16,-0.94,1,2,intermediate_model,b,False
0,MLP,2.10,0.81,1.36,0.26,2.99,0.53,0.25,0.55,3.44,3,0,non_dominant_model,b,True
3,LSTM_7.0,1.94,0.35,1.53,0.27,3.42,0.66,0.43,0.15,4.50,3,0,non_dominant_model,b,True


<IPython.core.display.Javascript object>